# Part One
Import required tools for collecting and cleaning the data: <br>
* Some tools are more commonly used
* Some tools are specific to the Alpha Vantage API

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv('secret')

In [2]:
import json
import requests
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import scipy.stats as sp
import pickle

In [3]:
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.timeseries import TimeSeries

import asyncio
from alpha_vantage.async_support.timeseries import TimeSeries

# Part Two
Obtain forex data from the Alpha Vantage API: <br>
* I am obtaining pricing data for the past 100 days for pairings of the top ten most traded currencies <br>
* For each pairing I am taking only the data required into a pandas dataframe, adding a column for "to" and "from" symbol to keep track of each currency, and then concatenating the dataframes into a large dataframe for easier use

In [4]:
app = ForeignExchange(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')

Create empty dataframe to concatenate to:

In [5]:
all_pairs = pd.DataFrame()

Obtain data from API:

In [6]:
usdeur = app.get_currency_exchange_daily('USD', 'EUR')

* Create dataframe from returned data
* Create new columns for "to" and "from" symbols
* Rename columns to clean up
* Show tail of dataframe in preparation of next step

In [7]:
usdeur1 = usdeur[0]
usdeur1['to symbol'] = 'EUR'
usdeur1['from symbol'] = 'USD'
usdeur1.rename(columns={'1. open': 'open'}, inplace=True)
usdeur1.rename(columns={'2. high': 'high'}, inplace=True)
usdeur1.rename(columns={'3. low': 'low'}, inplace=True)
usdeur1.rename(columns={'4. close': 'close'}, inplace=True)

usdeur1.tail()

,open,high,low,close,to symbol,from symbol
date,,,,,,
2022-05-19,0.9552,0.9553,0.9426,0.9445,EUR,USD
2022-05-18,0.9476,0.9558,0.9464,0.9555,EUR,USD
2022-05-17,0.9579,0.9586,0.9471,0.9477,EUR,USD
2022-05-16,0.9612,0.9622,0.9572,0.9579,EUR,USD
2022-05-13,0.9629,0.9658,0.9596,0.9600,EUR,USD


* Convert index to datetime
* Reorder index to ascending order
* Show head of dataframe to confirm that the reorder was successful

In [8]:
pd.to_datetime(usdeur1.index)
usdeur1 = usdeur1.sort_index()
usdeur1.head()

,open,high,low,close,to symbol,from symbol
date,,,,,,
2022-05-13,0.9629,0.9658,0.9596,0.9600,EUR,USD
2022-05-16,0.9612,0.9622,0.9572,0.9579,EUR,USD
2022-05-17,0.9579,0.9586,0.9471,0.9477,EUR,USD
2022-05-18,0.9476,0.9558,0.9464,0.9555,EUR,USD
2022-05-19,0.9552,0.9553,0.9426,0.9445,EUR,USD


* Create a time column in preparation for time series analysis
* Reorder columns to be more easily understood

In [9]:
usdeur1['time'] = np.arange(len(usdeur1.index))
usdeur1 = usdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

usdeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,EUR,0.9629,0.9658,0.9596,0.9600,0
2022-05-16,USD,EUR,0.9612,0.9622,0.9572,0.9579,1
2022-05-17,USD,EUR,0.9579,0.9586,0.9471,0.9477,2
2022-05-18,USD,EUR,0.9476,0.9558,0.9464,0.9555,3
2022-05-19,USD,EUR,0.9552,0.9553,0.9426,0.9445,4


Concatanate individual dataframe to all_pairs dataframe:

In [10]:
all_pairs = pd.concat([all_pairs, usdeur1])

Save as pickle for future use:

In [11]:
usdeur1.to_pickle('usdeur1.pkl')

Now repeat for all currency pairs:

In [12]:
usdjpy = app.get_currency_exchange_daily('USD', 'jpy')
usdjpy1 = usdjpy[0]
usdjpy1['to symbol'] = 'JPY'
usdjpy1['from symbol'] = 'USD'

usdjpy1.rename(columns={'1. open': 'open'}, inplace=True)
usdjpy1.rename(columns={'2. high': 'high'}, inplace=True)
usdjpy1.rename(columns={'3. low': 'low'}, inplace=True)
usdjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdjpy1.index)
usdjpy1 = usdjpy1.sort_index()

usdjpy1['time'] = np.arange(len(usdjpy1.index))
usdjpy1 = usdjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdjpy1])

usdjpy1.to_pickle('usdjpy1.pkl')

usdjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,JPY,128.407,129.454,128.32,129.185,0
2022-05-16,USD,JPY,129.350,129.640,128.67,129.062,1
2022-05-17,USD,JPY,129.057,129.780,128.80,129.360,2
2022-05-18,USD,JPY,129.352,129.536,127.98,128.305,3
2022-05-19,USD,JPY,128.301,128.944,126.99,127.713,4


In [13]:
usdgbp = app.get_currency_exchange_daily('USD', 'gbp')
usdgbp1 = usdgbp[0]
usdgbp1['to symbol'] = 'GBP'
usdgbp1['from symbol'] = 'USD'

usdgbp1.rename(columns={'1. open': 'open'}, inplace=True)
usdgbp1.rename(columns={'2. high': 'high'}, inplace=True)
usdgbp1.rename(columns={'3. low': 'low'}, inplace=True)
usdgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdgbp1.index)
usdgbp1 = usdgbp1.sort_index()

usdgbp1['time'] = np.arange(len(usdgbp1.index))
usdgbp1 = usdgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdgbp1])

usdgbp1.to_pickle('usdgbp1.pkl')

usdgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,GBP,0.81985,0.82239,0.81543,0.81550,0
2022-05-16,USD,GBP,0.81604,0.81856,0.81109,0.81154,1
2022-05-17,USD,GBP,0.81151,0.81184,0.80006,0.80046,2
2022-05-18,USD,GBP,0.80050,0.81115,0.79991,0.81031,3
2022-05-19,USD,GBP,0.81029,0.81045,0.79851,0.80160,4


In [14]:
usdaud = app.get_currency_exchange_daily('USD', 'aud')
usdaud1 = usdaud[0]
usdaud1['to symbol'] = 'AUD'
usdaud1['from symbol'] = 'USD'

usdaud1.rename(columns={'1. open': 'open'}, inplace=True)
usdaud1.rename(columns={'2. high': 'high'}, inplace=True)
usdaud1.rename(columns={'3. low': 'low'}, inplace=True)
usdaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdaud1.index)
usdaud1 = usdaud1.sort_index()

usdaud1['time'] = np.arange(len(usdaud1.index))
usdaud1 = usdaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdaud1])

usdaud1.to_pickle('usdaud1.pkl')

usdaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,AUD,1.45820,1.45844,1.44030,1.44030,0
2022-05-16,USD,AUD,1.44113,1.45476,1.43203,1.43390,1
2022-05-17,USD,AUD,1.43371,1.43484,1.42000,1.42211,2
2022-05-18,USD,AUD,1.42215,1.43871,1.41830,1.43645,3
2022-05-19,USD,AUD,1.43649,1.43767,1.41350,1.41840,4


In [15]:
usdchf = app.get_currency_exchange_daily('USD', 'chf')
usdchf1 = usdchf[0]
usdchf1['to symbol'] = 'CHF'
usdchf1['from symbol'] = 'USD'

usdchf1.rename(columns={'1. open': 'open'}, inplace=True)
usdchf1.rename(columns={'2. high': 'high'}, inplace=True)
usdchf1.rename(columns={'3. low': 'low'}, inplace=True)
usdchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdchf1.index)
usdchf1 = usdchf1.sort_index()

usdchf1['time'] = np.arange(len(usdchf1.index))
usdchf1 = usdchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdchf1])

usdchf1.to_pickle('usdchf1.pkl')

usdchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,CHF,1.00228,1.00468,0.9985,1.00117,0
2022-05-16,USD,CHF,1.00150,1.00643,1.0004,1.00130,1
2022-05-17,USD,CHF,1.00130,1.00237,0.9912,0.99280,2
2022-05-18,USD,CHF,0.99284,0.99850,0.9857,0.98814,3
2022-05-19,USD,CHF,0.98726,0.98870,0.9691,0.97154,4


In [16]:
usdcad = app.get_currency_exchange_daily('USD', 'cad')
usdcad1 = usdcad[0]
usdcad1['to symbol'] = 'CAD'
usdcad1['from symbol'] = 'USD'

usdcad1.rename(columns={'1. open': 'open'}, inplace=True)
usdcad1.rename(columns={'2. high': 'high'}, inplace=True)
usdcad1.rename(columns={'3. low': 'low'}, inplace=True)
usdcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdcad1.index)
usdcad1 = usdcad1.sort_index()

usdcad1['time'] = np.arange(len(usdcad1.index))
usdcad1 = usdcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdcad1])

usdcad1.to_pickle('usdcad1.pkl')

usdcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,CAD,1.30455,1.30480,1.28867,1.29080,0
2022-05-16,USD,CAD,1.29169,1.29814,1.28330,1.28493,1
2022-05-17,USD,CAD,1.28509,1.28587,1.28030,1.28072,2
2022-05-18,USD,CAD,1.28072,1.28958,1.27920,1.28840,3
2022-05-19,USD,CAD,1.28860,1.28886,1.27800,1.28220,4


In [17]:
usdhkd = app.get_currency_exchange_daily('USD', 'hkd')
usdhkd1 = usdhkd[0]
usdhkd1['to symbol'] = 'HKD'
usdhkd1['from symbol'] = 'USD'

usdhkd1.rename(columns={'1. open': 'open'}, inplace=True)
usdhkd1.rename(columns={'2. high': 'high'}, inplace=True)
usdhkd1.rename(columns={'3. low': 'low'}, inplace=True)
usdhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdhkd1.index)
usdhkd1 = usdhkd1.sort_index()

usdhkd1['time'] = np.arange(len(usdhkd1.index))
usdhkd1 = usdhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdhkd1])

usdhkd1.to_pickle('usdhkd1.pkl')

usdhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,HKD,7.84969,7.85030,7.84920,7.8498,0
2022-05-16,USD,HKD,7.84967,7.85002,7.84915,7.8499,1
2022-05-17,USD,HKD,7.84986,7.84994,7.84840,7.8491,2
2022-05-18,USD,HKD,7.84908,7.84975,7.84689,7.8488,3
2022-05-19,USD,HKD,7.84857,7.84914,7.84620,7.8475,4


In [18]:
usdsek = app.get_currency_exchange_daily('USD', 'sek')
usdsek1 = usdsek[0]
usdsek1['to symbol'] = 'SEK'
usdsek1['from symbol'] = 'USD'

usdsek1.rename(columns={'1. open': 'open'}, inplace=True)
usdsek1.rename(columns={'2. high': 'high'}, inplace=True)
usdsek1.rename(columns={'3. low': 'low'}, inplace=True)
usdsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdsek1.index)
usdsek1 = usdsek1.sort_index()

usdsek1['time'] = np.arange(len(usdsek1.index))
usdsek1 = usdsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdsek1])

usdsek1.to_pickle('usdsek1.pkl')

usdsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,SEK,10.15400,10.16320,10.04500,10.05710,0
2022-05-16,USD,SEK,10.06330,10.10241,10.02900,10.02900,1
2022-05-17,USD,SEK,10.05845,10.06927,9.87310,9.91695,2
2022-05-18,USD,SEK,9.91320,10.06420,9.90327,10.05275,3
2022-05-19,USD,SEK,10.05600,10.05690,9.87985,9.92620,4


In [19]:
usdnzd = app.get_currency_exchange_daily('USD', 'nzd')
usdnzd1 = usdnzd[0]
usdnzd1['to symbol'] = 'NZD'
usdnzd1['from symbol'] = 'USD'

usdnzd1.rename(columns={'1. open': 'open'}, inplace=True)
usdnzd1.rename(columns={'2. high': 'high'}, inplace=True)
usdnzd1.rename(columns={'3. low': 'low'}, inplace=True)
usdnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdnzd1.index)
usdnzd1 = usdnzd1.sort_index()

usdnzd1['time'] = np.arange(len(usdnzd1.index))
usdnzd1 = usdnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, usdnzd1])

usdnzd1.to_pickle('usdnzd1.pkl')

usdnzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,NZD,1.6041,1.6056,1.5895,1.5907,0
2022-05-16,USD,NZD,1.5908,1.6043,1.5821,1.5842,1
2022-05-17,USD,NZD,1.5840,1.5851,1.5681,1.5713,2
2022-05-18,USD,NZD,1.5718,1.5890,1.5692,1.5874,3
2022-05-19,USD,NZD,1.5876,1.5887,1.5585,1.5658,4


In [20]:
eurusd = app.get_currency_exchange_daily('EUR', 'USD')
eurusd1 = eurusd[0]
eurusd1['to symbol'] = 'USD'
eurusd1['from symbol'] = 'EUR'

eurusd1.rename(columns={'1. open': 'open'}, inplace=True)
eurusd1.rename(columns={'2. high': 'high'}, inplace=True)
eurusd1.rename(columns={'3. low': 'low'}, inplace=True)
eurusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurusd1.index)
eurusd1 = eurusd1.sort_index()

eurusd1['time'] = np.arange(len(eurusd1.index))
eurusd1 = eurusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurusd1])

eurusd1.to_pickle('eurusd1.pkl')

eurusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,USD,1.03788,1.04196,1.0348,1.04120,0
2022-05-16,EUR,USD,1.03968,1.04430,1.0387,1.04347,1
2022-05-17,EUR,USD,1.04350,1.05558,1.0427,1.05492,2
2022-05-18,EUR,USD,1.05500,1.05637,1.0458,1.04615,3
2022-05-19,EUR,USD,1.04617,1.06073,1.0458,1.05812,4


In [21]:
eurjpy = app.get_currency_exchange_daily('EUR', 'JPY')
eurjpy1 = eurjpy[0]
eurjpy1['to symbol'] = 'JPY'
eurjpy1['from symbol'] = 'EUR'

eurjpy1.rename(columns={'1. open': 'open'}, inplace=True)
eurjpy1.rename(columns={'2. high': 'high'}, inplace=True)
eurjpy1.rename(columns={'3. low': 'low'}, inplace=True)
eurjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurjpy1.index)
eurjpy1 = eurjpy1.sort_index()

eurjpy1['time'] = np.arange(len(eurjpy1.index))
eurjpy1 = eurjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurjpy1])

eurjpy1.to_pickle('eurjpy1.pkl')

eurjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,JPY,133.274,134.752,133.180,134.553,0
2022-05-16,EUR,JPY,134.484,135.049,133.710,134.691,1
2022-05-17,EUR,JPY,134.700,136.696,134.470,136.473,2
2022-05-18,EUR,JPY,136.473,136.669,133.991,134.210,3
2022-05-19,EUR,JPY,134.219,135.458,133.900,135.159,4


In [22]:
eurgbp = app.get_currency_exchange_daily('EUR', 'GBP')
eurgbp1 = eurgbp[0]
eurgbp1['to symbol'] = 'GBP'
eurgbp1['from symbol'] = 'EUR'

eurgbp1.rename(columns={'1. open': 'open'}, inplace=True)
eurgbp1.rename(columns={'2. high': 'high'}, inplace=True)
eurgbp1.rename(columns={'3. low': 'low'}, inplace=True)
eurgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurgbp1.index)
eurgbp1 = eurgbp1.sort_index()

eurgbp1['time'] = np.arange(len(eurgbp1.index))
eurgbp1 = eurgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurgbp1])

eurgbp1.to_pickle('eurgbp1.pkl')

eurgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,GBP,0.85090,0.85285,0.84837,0.84916,0
2022-05-16,EUR,GBP,0.84855,0.85338,0.84630,0.84681,1
2022-05-17,EUR,GBP,0.84680,0.84746,0.83910,0.84440,2
2022-05-18,EUR,GBP,0.84436,0.84942,0.84319,0.84755,3
2022-05-19,EUR,GBP,0.84754,0.84946,0.84460,0.84830,4


In [23]:
euraud = app.get_currency_exchange_daily('EUR', 'AUD')
euraud1 = euraud[0]
euraud1['to symbol'] = 'AUD'
euraud1['from symbol'] = 'EUR'

euraud1.rename(columns={'1. open': 'open'}, inplace=True)
euraud1.rename(columns={'2. high': 'high'}, inplace=True)
euraud1.rename(columns={'3. low': 'low'}, inplace=True)
euraud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(euraud1.index)
euraud1 = euraud1.sort_index()

euraud1['time'] = np.arange(len(euraud1.index))
euraud1 = euraud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, euraud1])

euraud1.to_pickle('euraud1.pkl')

euraud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,AUD,1.51340,1.51395,1.49834,1.49990,0
2022-05-16,EUR,AUD,1.49925,1.51184,1.49460,1.49623,1
2022-05-17,EUR,AUD,1.49620,1.50591,1.48810,1.50051,2
2022-05-18,EUR,AUD,1.50059,1.50571,1.49670,1.50363,3
2022-05-19,EUR,AUD,1.50345,1.50797,1.49440,1.50118,4


In [24]:
eurchf = app.get_currency_exchange_daily('EUR', 'CHF')
eurchf1 = eurchf[0]
eurchf1['to symbol'] = 'CHF'
eurchf1['from symbol'] = 'EUR'

eurchf1.rename(columns={'1. open': 'open'}, inplace=True)
eurchf1.rename(columns={'2. high': 'high'}, inplace=True)
eurchf1.rename(columns={'3. low': 'low'}, inplace=True)
eurchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurchf1.index)
eurchf1 = eurchf1.sort_index()

eurchf1['time'] = np.arange(len(eurchf1.index))
eurchf1 = eurchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurchf1])

eurchf1.to_pickle('eurchf1.pkl')

eurchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,CHF,1.04112,1.04370,1.03756,1.04278,0
2022-05-16,EUR,CHF,1.04207,1.04894,1.04020,1.04527,1
2022-05-17,EUR,CHF,1.04530,1.04850,1.04265,1.04800,2
2022-05-18,EUR,CHF,1.04767,1.04950,1.03279,1.03291,3
2022-05-19,EUR,CHF,1.03430,1.03602,1.02260,1.02951,4


In [25]:
eurcad = app.get_currency_exchange_daily('EUR', 'CAD')
eurcad1 = eurcad[0]
eurcad1['to symbol'] = 'CAD'
eurcad1['from symbol'] = 'EUR'

eurcad1.rename(columns={'1. open': 'open'}, inplace=True)
eurcad1.rename(columns={'2. high': 'high'}, inplace=True)
eurcad1.rename(columns={'3. low': 'low'}, inplace=True)
eurcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurcad1.index)
eurcad1 = eurcad1.sort_index()

eurcad1['time'] = np.arange(len(eurcad1.index))
eurcad1 = eurcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurcad1])

eurcad1.to_pickle('eurcad1.pkl')

eurcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,CAD,1.35405,1.35497,1.34104,1.34393,0
2022-05-16,EUR,CAD,1.34314,1.35019,1.33838,1.34088,1
2022-05-17,EUR,CAD,1.34087,1.35463,1.33858,1.35120,2
2022-05-18,EUR,CAD,1.35090,1.35249,1.34470,1.34783,3
2022-05-19,EUR,CAD,1.34806,1.35893,1.34481,1.35653,4


In [26]:
eurhkd = app.get_currency_exchange_daily('EUR', 'HKD')
eurhkd1 = eurhkd[0]
eurhkd1['to symbol'] = 'HKD'
eurhkd1['from symbol'] = 'EUR'

eurhkd1.rename(columns={'1. open': 'open'}, inplace=True)
eurhkd1.rename(columns={'2. high': 'high'}, inplace=True)
eurhkd1.rename(columns={'3. low': 'low'}, inplace=True)
eurhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurhkd1.index)
eurhkd1 = eurhkd1.sort_index()

eurhkd1['time'] = np.arange(len(eurhkd1.index))
eurhkd1 = eurhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurhkd1])

eurhkd1.to_pickle('eurhkd1.pkl')

eurhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,HKD,8.14810,8.17895,8.1238,8.17410,0
2022-05-16,EUR,HKD,8.16231,8.19740,8.1541,8.19010,1
2022-05-17,EUR,HKD,8.18930,8.28570,8.1848,8.27990,2
2022-05-18,EUR,HKD,8.28081,8.29138,8.2074,8.20989,3
2022-05-19,EUR,HKD,8.21141,8.32320,8.2110,8.30250,4


In [27]:
eursek = app.get_currency_exchange_daily('EUR', 'SEK')
eursek1 = eursek[0]
eursek1['to symbol'] = 'SEK'
eursek1['from symbol'] = 'EUR'

eursek1.rename(columns={'1. open': 'open'}, inplace=True)
eursek1.rename(columns={'2. high': 'high'}, inplace=True)
eursek1.rename(columns={'3. low': 'low'}, inplace=True)
eursek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eursek1.index)
eursek1 = eursek1.sort_index()

eursek1['time'] = np.arange(len(eursek1.index))
eursek1 = eursek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eursek1])

eursek1.to_pickle('eursek1.pkl')

eursek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,SEK,10.54133,10.54738,10.45978,10.47452,0
2022-05-16,EUR,SEK,10.47828,10.51582,10.46258,10.49338,1
2022-05-17,EUR,SEK,10.49774,10.51041,10.41130,10.46176,2
2022-05-18,EUR,SEK,10.46170,10.53411,10.44980,10.51933,3
2022-05-19,EUR,SEK,10.51303,10.52513,10.47065,10.50728,4


In [28]:
eurnzd = app.get_currency_exchange_daily('EUR', 'NZD')
eurnzd1 = eurnzd[0]
eurnzd1['to symbol'] = 'NZD'
eurnzd1['from symbol'] = 'EUR'

eurnzd1.rename(columns={'1. open': 'open'}, inplace=True)
eurnzd1.rename(columns={'2. high': 'high'}, inplace=True)
eurnzd1.rename(columns={'3. low': 'low'}, inplace=True)
eurnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurnzd1.index)
eurnzd1 = eurnzd1.sort_index()

eurnzd1['time'] = np.arange(len(eurnzd1.index))
eurnzd1 = eurnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, eurnzd1])

eurnzd1.to_pickle('eurnzd1.pkl')

eurnzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,EUR,NZD,1.66512,1.66654,1.65375,1.65651,0
2022-05-16,EUR,NZD,1.65427,1.66865,1.65005,1.65246,1
2022-05-17,EUR,NZD,1.65239,1.66227,1.64538,1.65794,2
2022-05-18,EUR,NZD,1.65814,1.66348,1.65210,1.66146,3
2022-05-19,EUR,NZD,1.66146,1.66678,1.64688,1.65650,4


In [29]:
gbpusd = app.get_currency_exchange_daily('GBP', 'USD')
gbpusd1 = gbpusd[0]
gbpusd1['to symbol'] = 'USD'
gbpusd1['from symbol'] = 'GBP'

gbpusd1.rename(columns={'1. open': 'open'}, inplace=True)
gbpusd1.rename(columns={'2. high': 'high'}, inplace=True)
gbpusd1.rename(columns={'3. low': 'low'}, inplace=True)
gbpusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpusd1.index)
gbpusd1 = gbpusd1.sort_index()

gbpusd1['time'] = np.arange(len(gbpusd1.index))
gbpusd1 = gbpusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpusd1])

gbpusd1.to_pickle('gbpusd1.pkl')

gbpusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,USD,1.21955,1.22628,1.2153,1.22610,0
2022-05-16,GBP,USD,1.22474,1.23294,1.2215,1.23195,1
2022-05-17,GBP,USD,1.23210,1.24988,1.2316,1.24892,2
2022-05-18,GBP,USD,1.24897,1.25009,1.2327,1.23387,3
2022-05-19,GBP,USD,1.23387,1.25246,1.2335,1.24721,4


In [30]:
gbpeur = app.get_currency_exchange_daily('GBP', 'EUR')
gbpeur1 = gbpeur[0]
gbpeur1['to symbol'] = 'EUR'
gbpeur1['from symbol'] = 'GBP'

gbpeur1.rename(columns={'1. open': 'open'}, inplace=True)
gbpeur1.rename(columns={'2. high': 'high'}, inplace=True)
gbpeur1.rename(columns={'3. low': 'low'}, inplace=True)
gbpeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpeur1.index)
gbpeur1 = gbpeur1.sort_index()

gbpeur1['time'] = np.arange(len(gbpeur1.index))
gbpeur1 = gbpeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpeur1])

gbpeur1.to_pickle('gbpeur1.pkl')

gbpeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,EUR,1.17466,1.17791,1.1722,1.1771,0
2022-05-16,GBP,EUR,1.17775,1.18104,1.1714,1.1803,1
2022-05-17,GBP,EUR,1.18016,1.19126,1.1795,1.1836,2
2022-05-18,GBP,EUR,1.18367,1.18547,1.1768,1.1790,3
2022-05-19,GBP,EUR,1.17888,1.18359,1.1767,1.1780,4


In [33]:
gbpjpy = app.get_currency_exchange_daily('GBP', 'JPY')
gbpjpy1 = gbpjpy[0]
gbpjpy1['to symbol'] = 'JPY'
gbpjpy1['from symbol'] = 'GBP'

gbpjpy1.rename(columns={'1. open': 'open'}, inplace=True)
gbpjpy1.rename(columns={'2. high': 'high'}, inplace=True)
gbpjpy1.rename(columns={'3. low': 'low'}, inplace=True)
gbpjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpjpy1.index)
gbpjpy1 = gbpjpy1.sort_index()

gbpjpy1['time'] = np.arange(len(gbpjpy1.index))
gbpjpy1 = gbpjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpjpy1])

gbpjpy1.to_pickle('gbpjpy1.pkl')

gbpjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,JPY,156.607,158.502,156.522,158.448,0
2022-05-16,GBP,JPY,158.427,159.278,157.400,158.970,1
2022-05-17,GBP,JPY,159.013,161.851,158.760,161.524,2
2022-05-18,GBP,JPY,161.563,161.784,157.995,158.317,3
2022-05-19,GBP,JPY,158.327,159.920,157.850,159.298,4


In [34]:
gbpaud = app.get_currency_exchange_daily('GBP', 'AUD')
gbpaud1 = gbpaud[0]
gbpaud1['to symbol'] = 'AUD'
gbpaud1['from symbol'] = 'GBP'

gbpaud1.rename(columns={'1. open': 'open'}, inplace=True)
gbpaud1.rename(columns={'2. high': 'high'}, inplace=True)
gbpaud1.rename(columns={'3. low': 'low'}, inplace=True)
gbpaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpaud1.index)
gbpaud1 = gbpaud1.sort_index()

gbpaud1['time'] = np.arange(len(gbpaud1.index))
gbpaud1 = gbpaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpaud1])

gbpaud1.to_pickle('gbpaud1.pkl')

gbpaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,AUD,1.77880,1.77946,1.76342,1.76683,0
2022-05-16,GBP,AUD,1.76603,1.77997,1.76331,1.76634,1
2022-05-17,GBP,AUD,1.76631,1.78212,1.76008,1.77666,2
2022-05-18,GBP,AUD,1.77666,1.78208,1.76587,1.77357,3
2022-05-19,GBP,AUD,1.77349,1.77685,1.76417,1.76848,4


In [35]:
gbpchf = app.get_currency_exchange_daily('GBP', 'CHF')
gbpchf1 = gbpchf[0]
gbpchf1['to symbol'] = 'CHF'
gbpchf1['from symbol'] = 'GBP'

gbpchf1.rename(columns={'1. open': 'open'}, inplace=True)
gbpchf1.rename(columns={'2. high': 'high'}, inplace=True)
gbpchf1.rename(columns={'3. low': 'low'}, inplace=True)
gbpchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpchf1.index)
gbpchf1 = gbpchf1.sort_index()

gbpchf1['time'] = np.arange(len(gbpchf1.index))
gbpchf1 = gbpchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpchf1])

gbpchf1.to_pickle('gbpchf1.pkl')

gbpchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,CHF,1.22309,1.22888,1.21790,1.22807,0
2022-05-16,GBP,CHF,1.22700,1.23504,1.22350,1.23390,1
2022-05-17,GBP,CHF,1.23394,1.24435,1.23311,1.24030,2
2022-05-18,GBP,CHF,1.24070,1.24196,1.21749,1.21895,3
2022-05-19,GBP,CHF,1.21908,1.22163,1.20650,1.21223,4


In [36]:
gbpcad = app.get_currency_exchange_daily('GBP', 'CAD')
gbpcad1 = gbpcad[0]
gbpcad1['to symbol'] = 'CAD'
gbpcad1['from symbol'] = 'GBP'

gbpcad1.rename(columns={'1. open': 'open'}, inplace=True)
gbpcad1.rename(columns={'2. high': 'high'}, inplace=True)
gbpcad1.rename(columns={'3. low': 'low'}, inplace=True)
gbpcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpcad1.index)
gbpcad1 = gbpcad1.sort_index()

gbpcad1['time'] = np.arange(len(gbpcad1.index))
gbpcad1 = gbpcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpcad1])

gbpcad1.to_pickle('gbpcad1.pkl')

gbpcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,CAD,1.59118,1.59145,1.57749,1.58296,0
2022-05-16,GBP,CAD,1.58221,1.58834,1.57956,1.58286,1
2022-05-17,GBP,CAD,1.58311,1.60304,1.58136,1.60002,2
2022-05-18,GBP,CAD,1.59971,1.60180,1.58707,1.58980,3
2022-05-19,GBP,CAD,1.59005,1.60398,1.58576,1.59950,4


In [38]:
gbphkd = app.get_currency_exchange_daily('GBP', 'HKD')
gbphkd1 = gbphkd[0]
gbphkd1['to symbol'] = 'HKD'
gbphkd1['from symbol'] = 'GBP'

gbphkd1.rename(columns={'1. open': 'open'}, inplace=True)
gbphkd1.rename(columns={'2. high': 'high'}, inplace=True)
gbphkd1.rename(columns={'3. low': 'low'}, inplace=True)
gbphkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbphkd1.index)
gbphkd1 = gbphkd1.sort_index()

gbphkd1['time'] = np.arange(len(gbphkd1.index))
gbphkd1 = gbphkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbphkd1])

gbphkd1.to_pickle('gbphkd1.pkl')

gbphkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,HKD,9.572,9.624,9.543,9.624,0
2022-05-16,GBP,HKD,9.616,9.675,9.590,9.669,1
2022-05-17,GBP,HKD,9.670,9.808,9.668,9.803,2
2022-05-18,GBP,HKD,9.803,9.808,9.677,9.684,3
2022-05-19,GBP,HKD,9.685,9.822,9.682,9.787,4


In [39]:
gbpsek = app.get_currency_exchange_daily('GBP', 'SEK')
gbpsek1 = gbpsek[0]
gbpsek1['to symbol'] = 'SEK'
gbpsek1['from symbol'] = 'GBP'

gbpsek1.rename(columns={'1. open': 'open'}, inplace=True)
gbpsek1.rename(columns={'2. high': 'high'}, inplace=True)
gbpsek1.rename(columns={'3. low': 'low'}, inplace=True)
gbpsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpsek1.index)
gbpsek1 = gbpsek1.sort_index()

gbpsek1['time'] = np.arange(len(gbpsek1.index))
gbpsek1 = gbpsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, gbpsek1])

gbpsek1.to_pickle('gbpsek1.pkl')

gbpsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,GBP,SEK,12.38140,12.39770,12.2813,12.3328,0
2022-05-16,GBP,SEK,12.28870,12.39996,12.2887,12.3877,1
2022-05-17,GBP,SEK,12.39505,12.42210,12.3238,12.3897,2
2022-05-18,GBP,SEK,12.38789,12.42495,12.3029,12.4054,3
2022-05-19,GBP,SEK,12.39698,12.42290,12.3494,12.3694,4


In [40]:
audusd = app.get_currency_exchange_daily('AUD', 'USD')
audusd1 = audusd[0]
audusd1['to symbol'] = 'USD'
audusd1['from symbol'] = 'AUD'

audusd1.rename(columns={'1. open': 'open'}, inplace=True)
audusd1.rename(columns={'2. high': 'high'}, inplace=True)
audusd1.rename(columns={'3. low': 'low'}, inplace=True)
audusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audusd1.index)
audusd1 = audusd1.sort_index()

audusd1['time'] = np.arange(len(audusd1.index))
audusd1 = audusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audusd1])

audusd1.to_pickle('audusd1.pkl')

audusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,USD,0.68549,0.69412,0.6850,0.69380,0
2022-05-16,AUD,USD,0.69319,0.69826,0.6870,0.69710,1
2022-05-17,AUD,USD,0.69706,0.70407,0.6965,0.70285,2
2022-05-18,AUD,USD,0.70281,0.70465,0.6946,0.69540,3
2022-05-19,AUD,USD,0.69544,0.70726,0.6950,0.70470,4


In [42]:
audeur = app.get_currency_exchange_daily('AUD', 'EUR')
audeur1 = audeur[0]
audeur1['to symbol'] = 'EUR'
audeur1['from symbol'] = 'AUD'

audeur1.rename(columns={'1. open': 'open'}, inplace=True)
audeur1.rename(columns={'2. high': 'high'}, inplace=True)
audeur1.rename(columns={'3. low': 'low'}, inplace=True)
audeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audeur1.index)
audeur1 = audeur1.sort_index()

audeur1['time'] = np.arange(len(audeur1.index))
audeur1 = audeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audeur1])

audeur1.to_pickle('audeur1.pkl')

audeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,EUR,0.66013,0.66670,0.66013,0.66630,0
2022-05-16,AUD,EUR,0.66575,0.66865,0.66124,0.66790,1
2022-05-17,AUD,EUR,0.66780,0.67150,0.66402,0.66589,2
2022-05-18,AUD,EUR,0.66604,0.66774,0.66390,0.66436,3
2022-05-19,AUD,EUR,0.66451,0.66871,0.66322,0.66570,4


In [43]:
audjpy = app.get_currency_exchange_daily('AUD', 'JPY')
audjpy1 = audjpy[0]
audjpy1['to symbol'] = 'JPY'
audjpy1['from symbol'] = 'AUD'

audjpy1.rename(columns={'1. open': 'open'}, inplace=True)
audjpy1.rename(columns={'2. high': 'high'}, inplace=True)
audjpy1.rename(columns={'3. low': 'low'}, inplace=True)
audjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audjpy1.index)
audjpy1 = audjpy1.sort_index()

audjpy1['time'] = np.arange(len(audjpy1.index))
audjpy1 = audjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audjpy1])

audjpy1.to_pickle('audjpy1.pkl')

audjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,JPY,88.017,89.718,87.893,89.675,0
2022-05-16,AUD,JPY,89.641,90.176,88.450,89.964,1
2022-05-17,AUD,JPY,89.972,91.168,89.770,90.913,2
2022-05-18,AUD,JPY,90.924,91.182,89.010,89.260,3
2022-05-19,AUD,JPY,89.252,90.531,88.980,90.055,4


In [44]:
audgbp = app.get_currency_exchange_daily('AUD', 'GBP')
audgbp1 = audgbp[0]
audgbp1['to symbol'] = 'GBP'
audgbp1['from symbol'] = 'AUD'

audgbp1.rename(columns={'1. open': 'open'}, inplace=True)
audgbp1.rename(columns={'2. high': 'high'}, inplace=True)
audgbp1.rename(columns={'3. low': 'low'}, inplace=True)
audgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audgbp1.index)
audgbp1 = audgbp1.sort_index()

audgbp1['time'] = np.arange(len(audgbp1.index))
audgbp1 = audgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audgbp1])

audgbp1.to_pickle('audgbp1.pkl')

audgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,GBP,0.56201,0.56678,0.56150,0.56550,0
2022-05-16,AUD,GBP,0.56447,0.56687,0.56151,0.56505,1
2022-05-17,AUD,GBP,0.56530,0.56778,0.56080,0.56220,2
2022-05-18,AUD,GBP,0.56230,0.56607,0.56060,0.56315,3
2022-05-19,AUD,GBP,0.56340,0.56654,0.56225,0.56243,4


In [45]:
audchf = app.get_currency_exchange_daily('AUD', 'CHF')
audchf1 = audchf[0]
audchf1['to symbol'] = 'CHF'
audchf1['from symbol'] = 'AUD'

audchf1.rename(columns={'1. open': 'open'}, inplace=True)
audchf1.rename(columns={'2. high': 'high'}, inplace=True)
audchf1.rename(columns={'3. low': 'low'}, inplace=True)
audchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audchf1.index)
audchf1 = audchf1.sort_index()

audchf1['time'] = np.arange(len(audchf1.index))
audchf1 = audchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audchf1])

audchf1.to_pickle('audchf1.pkl')

audchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,CHF,0.68725,0.69546,0.6866,0.69458,0
2022-05-16,AUD,CHF,0.69313,0.69925,0.6885,0.69820,1
2022-05-17,AUD,CHF,0.69815,0.70232,0.6935,0.69800,2
2022-05-18,AUD,CHF,0.69799,0.70046,0.6851,0.68690,3
2022-05-19,AUD,CHF,0.68674,0.69211,0.6799,0.68520,4


In [46]:
audcad = app.get_currency_exchange_daily('AUD', 'CAD')
audcad1 = audcad[0]
audcad1['to symbol'] = 'CAD'
audcad1['from symbol'] = 'AUD'

audcad1.rename(columns={'1. open': 'open'}, inplace=True)
audcad1.rename(columns={'2. high': 'high'}, inplace=True)
audcad1.rename(columns={'3. low': 'low'}, inplace=True)
audcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audcad1.index)
audcad1 = audcad1.sort_index()

audcad1['time'] = np.arange(len(audcad1.index))
audcad1 = audcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audcad1])

audcad1.to_pickle('audcad1.pkl')

audcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,CAD,0.89424,0.89751,0.89240,0.89505,0
2022-05-16,AUD,CAD,0.89557,0.89758,0.89139,0.89570,1
2022-05-17,AUD,CAD,0.89588,0.90272,0.89464,0.90006,2
2022-05-18,AUD,CAD,0.90017,0.90188,0.89479,0.89604,3
2022-05-19,AUD,CAD,0.89615,0.90529,0.89409,0.90360,4


In [47]:
audhkd = app.get_currency_exchange_daily('AUD', 'HKD')
audhkd1 = audhkd[0]
audhkd1['to symbol'] = 'HKD'
audhkd1['from symbol'] = 'AUD'

audhkd1.rename(columns={'1. open': 'open'}, inplace=True)
audhkd1.rename(columns={'2. high': 'high'}, inplace=True)
audhkd1.rename(columns={'3. low': 'low'}, inplace=True)
audhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audhkd1.index)
audhkd1 = audhkd1.sort_index()

audhkd1['time'] = np.arange(len(audhkd1.index))
audhkd1 = audhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audhkd1])

audhkd1.to_pickle('audhkd1.pkl')

audhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,HKD,5.3802,5.4477,5.3755,5.4458,0
2022-05-16,AUD,HKD,5.4444,5.4807,5.3825,5.4719,1
2022-05-17,AUD,HKD,5.4722,5.5261,5.4580,5.5171,2
2022-05-18,AUD,HKD,5.5176,5.5303,5.4532,5.4585,3
2022-05-19,AUD,HKD,5.4609,5.5488,5.4455,5.5293,4


In [48]:
audsek = app.get_currency_exchange_daily('AUD', 'SEK')
audsek1 = audsek[0]
audsek1['to symbol'] = 'SEK'
audsek1['from symbol'] = 'AUD'

audsek1.rename(columns={'1. open': 'open'}, inplace=True)
audsek1.rename(columns={'2. high': 'high'}, inplace=True)
audsek1.rename(columns={'3. low': 'low'}, inplace=True)
audsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audsek1.index)
audsek1 = audsek1.sort_index()

audsek1['time'] = np.arange(len(audsek1.index))
audsek1 = audsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audsek1])

audsek1.to_pickle('audsek1.pkl')

audsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,SEK,6.9603,6.9824,6.9415,6.9778,0
2022-05-16,AUD,SEK,6.9748,7.0137,6.9233,7.0067,1
2022-05-17,AUD,SEK,7.0076,7.0381,6.9393,6.9577,2
2022-05-18,AUD,SEK,6.9604,7.0159,6.9558,6.9854,3
2022-05-19,AUD,SEK,6.9804,7.0153,6.9638,6.9826,4


In [49]:
audnzd = app.get_currency_exchange_daily('AUD', 'NZD')
audnzd1 = audnzd[0]
audnzd1['to symbol'] = 'NZD'
audnzd1['from symbol'] = 'AUD'

audnzd1.rename(columns={'1. open': 'open'}, inplace=True)
audnzd1.rename(columns={'2. high': 'high'}, inplace=True)
audnzd1.rename(columns={'3. low': 'low'}, inplace=True)
audnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audnzd1.index)
audnzd1 = audnzd1.sort_index()

audnzd1['time'] = np.arange(len(audnzd1.index))
audnzd1 = audnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, audnzd1])

audnzd1.to_pickle('audnzd1.pkl')

audnzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,AUD,NZD,1.09942,1.10796,1.09839,1.10388,0
2022-05-16,AUD,NZD,1.10304,1.10674,1.10100,1.10436,1
2022-05-17,AUD,NZD,1.10357,1.10801,1.10052,1.10453,2
2022-05-18,AUD,NZD,1.10455,1.10639,1.10100,1.10446,3
2022-05-19,AUD,NZD,1.10452,1.10698,1.09875,1.10354,4


In [50]:
chfusd = app.get_currency_exchange_daily('CHF', 'USD')
chfusd1 = chfusd[0]
chfusd1['to symbol'] = 'USD'
chfusd1['from symbol'] = 'CHF'

chfusd1.rename(columns={'1. open': 'open'}, inplace=True)
chfusd1.rename(columns={'2. high': 'high'}, inplace=True)
chfusd1.rename(columns={'3. low': 'low'}, inplace=True)
chfusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfusd1.index)
chfusd1 = chfusd1.sort_index()

chfusd1['time'] = np.arange(len(chfusd1.index))
chfusd1 = chfusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfusd1])

chfusd1.to_pickle('chfusd1.pkl')

chfusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,USD,0.99641,1.00062,0.9951,0.99770,0
2022-05-16,CHF,USD,0.99615,0.99903,0.9931,0.99570,1
2022-05-17,CHF,USD,0.99706,1.00808,0.9970,1.00547,2
2022-05-18,CHF,USD,1.00575,1.01388,1.0011,1.00907,3
2022-05-19,CHF,USD,1.01103,1.03100,1.0110,1.02740,4


In [51]:
chfeur = app.get_currency_exchange_daily('CHF', 'EUR')
chfeur1 = chfeur[0]
chfeur1['to symbol'] = 'EUR'
chfeur1['from symbol'] = 'CHF'

chfeur1.rename(columns={'1. open': 'open'}, inplace=True)
chfeur1.rename(columns={'2. high': 'high'}, inplace=True)
chfeur1.rename(columns={'3. low': 'low'}, inplace=True)
chfeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfeur1.index)
chfeur1 = chfeur1.sort_index()

chfeur1['time'] = np.arange(len(chfeur1.index))
chfeur1 = chfeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfeur1])

chfeur1.to_pickle('chfeur1.pkl')

chfeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,EUR,0.9596,0.9628,0.9576,0.9580,0
2022-05-16,CHF,EUR,0.9586,0.9604,0.9528,0.9556,1
2022-05-17,CHF,EUR,0.9560,0.9581,0.9531,0.9536,2
2022-05-18,CHF,EUR,0.9532,0.9673,0.9523,0.9672,3
2022-05-19,CHF,EUR,0.9661,0.9768,0.9648,0.9704,4


In [52]:
chfjpy = app.get_currency_exchange_daily('CHF', 'JPY')
chfjpy1 = chfjpy[0]
chfjpy1['to symbol'] = 'JPY'
chfjpy1['from symbol'] = 'CHF'

chfjpy1.rename(columns={'1. open': 'open'}, inplace=True)
chfjpy1.rename(columns={'2. high': 'high'}, inplace=True)
chfjpy1.rename(columns={'3. low': 'low'}, inplace=True)
chfjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfjpy1.index)
chfjpy1 = chfjpy1.sort_index()

chfjpy1['time'] = np.arange(len(chfjpy1.index))
chfjpy1 = chfjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfjpy1])

chfjpy1.to_pickle('chfjpy1.pkl')

chfjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,JPY,128.020,129.222,127.865,128.971,0
2022-05-16,CHF,JPY,129.016,129.274,128.240,128.769,1
2022-05-17,CHF,JPY,128.800,130.608,128.600,130.180,2
2022-05-18,CHF,JPY,130.148,130.377,129.020,129.921,3
2022-05-19,CHF,JPY,129.696,131.640,129.540,131.442,4


In [53]:
chfgbp = app.get_currency_exchange_daily('CHF', 'GBP')
chfgbp1 = chfgbp[0]
chfgbp1['to symbol'] = 'GBP'
chfgbp1['from symbol'] = 'CHF'

chfgbp1.rename(columns={'1. open': 'open'}, inplace=True)
chfgbp1.rename(columns={'2. high': 'high'}, inplace=True)
chfgbp1.rename(columns={'3. low': 'low'}, inplace=True)
chfgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfgbp1.index)
chfgbp1 = chfgbp1.sort_index()

chfgbp1['time'] = np.arange(len(chfgbp1.index))
chfgbp1 = chfgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfgbp1])

chfgbp1.to_pickle('chfgbp1.pkl')

chfgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,GBP,0.8168,0.8202,0.8131,0.8135,0
2022-05-16,CHF,GBP,0.8136,0.8158,0.8091,0.8092,1
2022-05-17,CHF,GBP,0.8096,0.8097,0.8033,0.8053,2
2022-05-18,CHF,GBP,0.8050,0.8200,0.8046,0.8199,3
2022-05-19,CHF,GBP,0.8192,0.8278,0.8181,0.8233,4


In [55]:
chfaud = app.get_currency_exchange_daily('CHF', 'AUD')
chfaud1 = chfaud[0]
chfaud1['to symbol'] = 'AUD'
chfaud1['from symbol'] = 'CHF'

chfaud1.rename(columns={'1. open': 'open'}, inplace=True)
chfaud1.rename(columns={'2. high': 'high'}, inplace=True)
chfaud1.rename(columns={'3. low': 'low'}, inplace=True)
chfaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfaud1.index)
chfaud1 = chfaud1.sort_index()
chfaud1
chfaud1['time'] = np.arange(len(chfaud1.index))
chfaud1 = chfaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfaud1])

chfaud1.to_pickle('chfaud1.pkl')

chfaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,AUD,1.45149,1.45410,1.43641,1.4377,0
2022-05-16,CHF,AUD,1.43645,1.44976,1.42900,1.4306,1
2022-05-17,CHF,AUD,1.43100,1.43920,1.42350,1.4314,2
2022-05-18,CHF,AUD,1.43100,1.45550,1.42720,1.4546,3
2022-05-19,CHF,AUD,1.45310,1.46840,1.44457,1.4577,4


In [56]:
chfcad = app.get_currency_exchange_daily('CHF', 'CAD')
chfcad1 = chfcad[0]
chfcad1['to symbol'] = 'CAD'
chfcad1['from symbol'] = 'CHF'

chfcad1.rename(columns={'1. open': 'open'}, inplace=True)
chfcad1.rename(columns={'2. high': 'high'}, inplace=True)
chfcad1.rename(columns={'3. low': 'low'}, inplace=True)
chfcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfcad1.index)
chfcad1 = chfcad1.sort_index()

chfcad1['time'] = np.arange(len(chfcad1.index))
chfcad1 = chfcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfcad1])

chfcad1.to_pickle('chfcad1.pkl')

chfcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,CAD,1.2996,1.3006,1.2869,1.2877,0
2022-05-16,CHF,CAD,1.2874,1.2930,1.2800,1.2815,1
2022-05-17,CHF,CAD,1.2816,1.2938,1.2791,1.2883,2
2022-05-18,CHF,CAD,1.2881,1.3044,1.2836,1.3026,3
2022-05-19,CHF,CAD,1.3029,1.3192,1.2995,1.3168,4


In [58]:
chfhkd = app.get_currency_exchange_daily('CHF', 'HKD')
chfhkd1 = chfhkd[0]
chfhkd1['to symbol'] = 'HKD'
chfhkd1['from symbol'] = 'CHF'

chfhkd1.rename(columns={'1. open': 'open'}, inplace=True)
chfhkd1.rename(columns={'2. high': 'high'}, inplace=True)
chfhkd1.rename(columns={'3. low': 'low'}, inplace=True)
chfhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfhkd1.index)
chfhkd1 = chfhkd1.sort_index()

chfhkd1['time'] = np.arange(len(chfhkd1.index))
chfhkd1 = chfhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfhkd1])

chfhkd1.to_pickle('chfhkd1.pkl')

chfhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,HKD,7.8253,7.8555,7.7955,7.8350,0
2022-05-16,CHF,HKD,7.8300,7.8429,7.7830,7.8318,1
2022-05-17,CHF,HKD,7.8348,7.9137,7.8115,7.9004,2
2022-05-18,CHF,HKD,7.8973,7.9564,7.8465,7.9463,3
2022-05-19,CHF,HKD,7.9400,8.0919,7.9295,8.0641,4


In [60]:
chfsek = app.get_currency_exchange_daily('CHF', 'SEK')
chfsek1 = chfsek[0]
chfsek1['to symbol'] = 'SEK'
chfsek1['from symbol'] = 'CHF'

chfsek1.rename(columns={'1. open': 'open'}, inplace=True)
chfsek1.rename(columns={'2. high': 'high'}, inplace=True)
chfsek1.rename(columns={'3. low': 'low'}, inplace=True)
chfsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfsek1.index)
chfsek1 = chfsek1.sort_index()

chfsek1['time'] = np.arange(len(chfsek1.index))
chfsek1 = chfsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfsek1])

chfsek1.to_pickle('chfsek1.pkl')

chfsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,SEK,10.1201,10.13130,10.0186,10.0419,0
2022-05-16,CHF,SEK,10.0374,10.08640,9.8123,10.0083,1
2022-05-17,CHF,SEK,10.0256,10.05430,9.9455,9.9722,2
2022-05-18,CHF,SEK,9.9837,10.18305,9.9547,10.1812,3
2022-05-19,CHF,SEK,10.1715,10.26671,10.1191,10.2030,4


In [61]:
chfnzd = app.get_currency_exchange_daily('CHF', 'NZD')
chfnzd1 = chfnzd[0]
chfnzd1['to symbol'] = 'NZD'
chfnzd1['from symbol'] = 'CHF'

chfnzd1.rename(columns={'1. open': 'open'}, inplace=True)
chfnzd1.rename(columns={'2. high': 'high'}, inplace=True)
chfnzd1.rename(columns={'3. low': 'low'}, inplace=True)
chfnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfnzd1.index)
chfnzd1 = chfnzd1.sort_index()

chfnzd1['time'] = np.arange(len(chfnzd1.index))
chfnzd1 = chfnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, chfnzd1])

chfnzd1.to_pickle('chfnzd1.pkl')

chfnzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CHF,NZD,1.5991,1.6034,1.5855,1.5878,0
2022-05-16,CHF,NZD,1.5869,1.5995,1.5794,1.5806,1
2022-05-17,CHF,NZD,1.5810,1.5888,1.5747,1.5817,2
2022-05-18,CHF,NZD,1.5816,1.6078,1.5757,1.6072,3
2022-05-19,CHF,NZD,1.6061,1.6185,1.5984,1.6091,4


In [63]:
cadusd = app.get_currency_exchange_daily('CAD', 'USD')
cadusd1 = cadusd[0]
cadusd1['to symbol'] = 'USD'
cadusd1['from symbol'] = 'CAD'

cadusd1.rename(columns={'1. open': 'open'}, inplace=True)
cadusd1.rename(columns={'2. high': 'high'}, inplace=True)
cadusd1.rename(columns={'3. low': 'low'}, inplace=True)
cadusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadusd1.index)
cadusd1 = cadusd1.sort_index()

cadusd1['time'] = np.arange(len(cadusd1.index))
cadusd1 = cadusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadusd1])

chfnzd1.to_pickle('chfnzd1.pkl')

cadusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,USD,0.76632,0.77468,0.7659,0.77410,0
2022-05-16,CAD,USD,0.77363,0.77895,0.7698,0.77760,1
2022-05-17,CAD,USD,0.77790,0.78077,0.7773,0.78048,2
2022-05-18,CAD,USD,0.78051,0.78140,0.7748,0.77550,3
2022-05-19,CAD,USD,0.77576,0.78210,0.7754,0.77920,4


In [64]:
cadeur = app.get_currency_exchange_daily('CAD', 'EUR')
cadeur1 = cadeur[0]
cadeur1['to symbol'] = 'EUR'
cadeur1['from symbol'] = 'CAD'

cadeur1.rename(columns={'1. open': 'open'}, inplace=True)
cadeur1.rename(columns={'2. high': 'high'}, inplace=True)
cadeur1.rename(columns={'3. low': 'low'}, inplace=True)
cadeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadeur1.index)
cadeur1 = cadeur1.sort_index()

cadeur1['time'] = np.arange(len(cadeur1.index))
cadeur1 = cadeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadeur1])

cadeur1.to_pickle('cadeur1.pkl')

cadeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,EUR,0.7380,0.7447,0.7380,0.7436,0
2022-05-16,CAD,EUR,0.7444,0.7463,0.7404,0.7453,1
2022-05-17,CAD,EUR,0.7453,0.7465,0.7382,0.7397,2
2022-05-18,CAD,EUR,0.7396,0.7432,0.7391,0.7414,3
2022-05-19,CAD,EUR,0.7413,0.7429,0.7357,0.7365,4


In [65]:
cadjpy = app.get_currency_exchange_daily('CAD', 'JPY')
cadjpy1 = cadjpy[0]
cadjpy1['to symbol'] = 'JPY'
cadjpy1['from symbol'] = 'CAD'

cadjpy1.rename(columns={'1. open': 'open'}, inplace=True)
cadjpy1.rename(columns={'2. high': 'high'}, inplace=True)
cadjpy1.rename(columns={'3. low': 'low'}, inplace=True)
cadjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadjpy1.index)
cadjpy1 = cadjpy1.sort_index()

cadjpy1['time'] = np.arange(len(cadjpy1.index))
cadjpy1 = cadjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadjpy1])

cadjpy1.to_pickle('cadjpy1.pkl')

cadjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,JPY,98.398,100.236,98.365,100.058,0
2022-05-16,CAD,JPY,100.096,100.690,99.160,100.395,1
2022-05-17,CAD,JPY,100.409,101.118,100.260,100.975,2
2022-05-18,CAD,JPY,100.971,101.155,99.361,99.549,3
2022-05-19,CAD,JPY,99.550,100.568,99.110,99.590,4


In [66]:
cadgbp = app.get_currency_exchange_daily('CAD', 'GBP')
cadgbp1 = cadgbp[0]
cadgbp1['to symbol'] = 'GBP'
cadgbp1['from symbol'] = 'CAD'

cadgbp1.rename(columns={'1. open': 'open'}, inplace=True)
cadgbp1.rename(columns={'2. high': 'high'}, inplace=True)
cadgbp1.rename(columns={'3. low': 'low'}, inplace=True)
cadgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadgbp1.index)
cadgbp1 = cadgbp1.sort_index()

cadgbp1['time'] = np.arange(len(cadgbp1.index))
cadgbp1 = cadgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadgbp1])

cadgbp1.to_pickle('cadgbp1.pkl')

cadgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,GBP,0.6279,0.6333,0.6278,0.6311,0
2022-05-16,CAD,GBP,0.6316,0.6325,0.6291,0.6309,1
2022-05-17,CAD,GBP,0.6311,0.6316,0.6234,0.6245,2
2022-05-18,CAD,GBP,0.6245,0.6295,0.6238,0.6283,3
2022-05-19,CAD,GBP,0.6283,0.6299,0.6231,0.6246,4


In [67]:
cadaud = app.get_currency_exchange_daily('CAD', 'AUD')
cadaud1 = cadaud[0]
cadaud1['to symbol'] = 'AUD'
cadaud1['from symbol'] = 'CAD'

cadaud1.rename(columns={'1. open': 'open'}, inplace=True)
cadaud1.rename(columns={'2. high': 'high'}, inplace=True)
cadaud1.rename(columns={'3. low': 'low'}, inplace=True)
cadaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadaud1.index)
cadaud1 = cadaud1.sort_index()

cadaud1['time'] = np.arange(len(cadaud1.index))
cadaud1 = cadaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadaud1])

cadaud1.to_pickle('cadaud1.pkl')

cadaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,AUD,1.1170,1.1188,1.1137,1.1155,0
2022-05-16,CAD,AUD,1.1156,1.1201,1.1139,1.1156,1
2022-05-17,CAD,AUD,1.1157,1.1160,1.1074,1.1102,2
2022-05-18,CAD,AUD,1.1102,1.1158,1.1080,1.1148,3
2022-05-19,CAD,AUD,1.1146,1.1170,1.1040,1.1059,4


In [68]:
cadchf = app.get_currency_exchange_daily('CAD', 'CHF')
cadchf1 = cadchf[0]
cadchf1['to symbol'] = 'CHF'
cadchf1['from symbol'] = 'CAD'

cadchf1.rename(columns={'1. open': 'open'}, inplace=True)
cadchf1.rename(columns={'2. high': 'high'}, inplace=True)
cadchf1.rename(columns={'3. low': 'low'}, inplace=True)
cadchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadchf1.index)
cadchf1 = cadchf1.sort_index()

cadchf1['time'] = np.arange(len(cadchf1.index))
cadchf1 = cadchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadchf1])

cadchf1.to_pickle('cadchf1.pkl')

cadchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,CHF,0.76823,0.77712,0.76750,0.77525,0
2022-05-16,CAD,CHF,0.77448,0.78064,0.77100,0.77939,1
2022-05-17,CAD,CHF,0.77921,0.78123,0.77110,0.77513,2
2022-05-18,CAD,CHF,0.77537,0.77887,0.76516,0.76612,3
2022-05-19,CAD,CHF,0.76585,0.76901,0.75609,0.75750,4


In [69]:
cadhkd = app.get_currency_exchange_daily('CAD', 'HKD')
cadhkd1 = cadhkd[0]
cadhkd1['to symbol'] = 'HKD'
cadhkd1['from symbol'] = 'CAD'

cadhkd1.rename(columns={'1. open': 'open'}, inplace=True)
cadhkd1.rename(columns={'2. high': 'high'}, inplace=True)
cadhkd1.rename(columns={'3. low': 'low'}, inplace=True)
cadhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadhkd1.index)
cadhkd1 = cadhkd1.sort_index()

cadhkd1['time'] = np.arange(len(cadhkd1.index))
cadhkd1 = cadhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadhkd1])

cadhkd1.to_pickle('cadhkd1.pkl')

cadhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,HKD,6.01554,6.08438,6.0080,6.07880,0
2022-05-16,CAD,HKD,6.07454,6.11485,6.0305,6.10654,1
2022-05-17,CAD,HKD,6.10759,6.12889,6.0895,6.12736,2
2022-05-18,CAD,HKD,6.12764,6.13422,6.0856,6.08943,3
2022-05-19,CAD,HKD,6.08977,6.13829,6.0740,6.11724,4


In [70]:
cadsek = app.get_currency_exchange_daily('CAD', 'SEK')
cadsek1 = cadsek[0]
cadsek1['to symbol'] = 'SEK'
cadsek1['from symbol'] = 'CAD'

cadsek1.rename(columns={'1. open': 'open'}, inplace=True)
cadsek1.rename(columns={'2. high': 'high'}, inplace=True)
cadsek1.rename(columns={'3. low': 'low'}, inplace=True)
cadsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadsek1.index)
cadsek1 = cadsek1.sort_index()

cadsek1['time'] = np.arange(len(cadsek1.index))
cadsek1 = cadsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadsek1])

cadsek1.to_pickle('cadsek1.pkl')

cadsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,SEK,7.7358,7.8009,7.7350,7.7879,0
2022-05-16,CAD,SEK,7.7833,7.8343,7.7526,7.8125,1
2022-05-17,CAD,SEK,7.8133,7.8351,7.6980,7.7310,2
2022-05-18,CAD,SEK,7.7314,7.8152,7.7291,7.7962,3
2022-05-19,CAD,SEK,7.7851,7.8095,7.7055,7.7145,4


In [71]:
cadnzd = app.get_currency_exchange_daily('CAD', 'NZD')
cadnzd1 = cadnzd[0]
cadnzd1['to symbol'] = 'NZD'
cadnzd1['from symbol'] = 'CAD'

cadnzd1.rename(columns={'1. open': 'open'}, inplace=True)
cadnzd1.rename(columns={'2. high': 'high'}, inplace=True)
cadnzd1.rename(columns={'3. low': 'low'}, inplace=True)
cadnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadnzd1.index)
cadnzd1 = cadnzd1.sort_index()

cadnzd1['time'] = np.arange(len(cadnzd1.index))
cadnzd1 = cadnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, cadnzd1])

cadnzd1.to_pickle('cadnzd1.pkl')

cadnzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,CAD,NZD,1.2284,1.2349,1.2276,1.2312,0
2022-05-16,CAD,NZD,1.2309,1.2362,1.2284,1.2319,1
2022-05-17,CAD,NZD,1.2316,1.2333,1.2229,1.2257,2
2022-05-18,CAD,NZD,1.2258,1.2319,1.2235,1.2316,3
2022-05-19,CAD,NZD,1.2311,1.2334,1.2172,1.2209,4


In [72]:
hkdusd = app.get_currency_exchange_daily('HKD', 'USD')
hkdusd1 = hkdusd[0]
hkdusd1['to symbol'] = 'USD'
hkdusd1['from symbol'] = 'HKD'

hkdusd1.rename(columns={'1. open': 'open'}, inplace=True)
hkdusd1.rename(columns={'2. high': 'high'}, inplace=True)
hkdusd1.rename(columns={'3. low': 'low'}, inplace=True)
hkdusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdusd1.index)
hkdusd1 = hkdusd1.sort_index()

hkdusd1['time'] = np.arange(len(hkdusd1.index))
hkdusd1 = hkdusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdusd1])

hkdusd1.to_pickle('hkdusd1.pkl')

hkdusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-12,HKD,USD,0.12739,0.12740,0.12739,0.12739,0
2022-05-15,HKD,USD,0.12739,0.12740,0.12739,0.12739,1
2022-05-16,HKD,USD,0.12739,0.12741,0.12739,0.12739,2
2022-05-17,HKD,USD,0.12740,0.12744,0.12739,0.12740,3
2022-05-18,HKD,USD,0.12741,0.12745,0.12740,0.12741,4


In [73]:
hkdeur = app.get_currency_exchange_daily('HKD', 'EUR')
hkdeur1 = hkdeur[0]
hkdeur1['to symbol'] = 'EUR'
hkdeur1['from symbol'] = 'HKD'

hkdeur1.rename(columns={'1. open': 'open'}, inplace=True)
hkdeur1.rename(columns={'2. high': 'high'}, inplace=True)
hkdeur1.rename(columns={'3. low': 'low'}, inplace=True)
hkdeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdeur1.index)
hkdeur1 = hkdeur1.sort_index()

hkdeur1['time'] = np.arange(len(hkdeur1.index))
hkdeur1 = hkdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdeur1])

hkdeur1.to_pickle('hkdeur1.pkl')

hkdeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,EUR,0.12265,0.12295,0.12222,0.12228,0
2022-05-16,HKD,EUR,0.12242,0.12254,0.12194,0.12201,1
2022-05-17,HKD,EUR,0.12202,0.12209,0.12067,0.12069,2
2022-05-18,HKD,EUR,0.12071,0.12173,0.12056,0.12172,3
2022-05-19,HKD,EUR,0.12169,0.12169,0.12011,0.12035,4


In [74]:
hkdjpy = app.get_currency_exchange_daily('HKD', 'JPY')
hkdjpy1 = hkdjpy[0]
hkdjpy1['to symbol'] = 'JPY'
hkdjpy1['from symbol'] = 'HKD'

hkdjpy1.rename(columns={'1. open': 'open'}, inplace=True)
hkdjpy1.rename(columns={'2. high': 'high'}, inplace=True)
hkdjpy1.rename(columns={'3. low': 'low'}, inplace=True)
hkdjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdjpy1.index)
hkdjpy1 = hkdjpy1.sort_index()

hkdjpy1['time'] = np.arange(len(hkdjpy1.index))
hkdjpy1 = hkdjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdjpy1])

hkdjpy1.to_pickle('hkdjpy1.pkl')

hkdjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,JPY,16.36095,16.49020,16.34940,16.45970,0
2022-05-16,HKD,JPY,16.45870,16.51418,16.37482,16.44018,1
2022-05-17,HKD,JPY,16.44285,16.53320,16.41073,16.47870,2
2022-05-18,HKD,JPY,16.48166,16.50206,16.30797,16.34616,3
2022-05-19,HKD,JPY,16.34781,16.42996,16.18740,16.27404,4


In [75]:
hkdgbp = app.get_currency_exchange_daily('HKD', 'GBP')
hkdgbp1 = hkdgbp[0]
hkdgbp1['to symbol'] = 'GBP'
hkdgbp1['from symbol'] = 'HKD'

hkdgbp1.rename(columns={'1. open': 'open'}, inplace=True)
hkdgbp1.rename(columns={'2. high': 'high'}, inplace=True)
hkdgbp1.rename(columns={'3. low': 'low'}, inplace=True)
hkdgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdgbp1.index)
hkdgbp1 = hkdgbp1.sort_index()

hkdgbp1['time'] = np.arange(len(hkdgbp1.index))
hkdgbp1 = hkdgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdgbp1])

hkdgbp1.to_pickle('hkdgbp1.pkl')

hkdgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,GBP,0.10439,0.10467,0.10383,0.10383,0
2022-05-16,HKD,GBP,0.10392,0.10420,0.10328,0.10334,1
2022-05-17,HKD,GBP,0.10334,0.10336,0.10192,0.10194,2
2022-05-18,HKD,GBP,0.10194,0.10326,0.10187,0.10319,3
2022-05-19,HKD,GBP,0.10319,0.10319,0.10174,0.10210,4


In [76]:
hkdaud = app.get_currency_exchange_daily('HKD', 'AUD')
hkdaud1 = hkdaud[0]
hkdaud1['to symbol'] = 'AUD'
hkdaud1['from symbol'] = 'HKD'

hkdaud1.rename(columns={'1. open': 'open'}, inplace=True)
hkdaud1.rename(columns={'2. high': 'high'}, inplace=True)
hkdaud1.rename(columns={'3. low': 'low'}, inplace=True)
hkdaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdaud1.index)
hkdaud1 = hkdaud1.sort_index()

hkdaud1['time'] = np.arange(len(hkdaud1.index))
hkdaud1 = hkdaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdaud1])

hkdaud1.to_pickle('hkdaud1.pkl')

hkdaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,AUD,0.18569,0.18575,0.18348,0.18349,0
2022-05-16,HKD,AUD,0.18352,0.18521,0.18246,0.18267,1
2022-05-17,HKD,AUD,0.18265,0.18274,0.18091,0.18119,2
2022-05-18,HKD,AUD,0.18121,0.18327,0.18071,0.18306,3
2022-05-19,HKD,AUD,0.18301,0.18314,0.18013,0.18076,4


In [77]:
hkdchf = app.get_currency_exchange_daily('HKD', 'CHF')
hkdchf1 = hkdchf[0]
hkdchf1['to symbol'] = 'CHF'
hkdchf1['from symbol'] = 'HKD'

hkdchf1.rename(columns={'1. open': 'open'}, inplace=True)
hkdchf1.rename(columns={'2. high': 'high'}, inplace=True)
hkdchf1.rename(columns={'3. low': 'low'}, inplace=True)
hkdchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdchf1.index)
hkdchf1 = hkdchf1.sort_index()

hkdchf1['time'] = np.arange(len(hkdchf1.index))
hkdchf1 = hkdchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdchf1])

hkdchf1.to_pickle('hkdchf1.pkl')

hkdchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,CHF,0.12773,0.12796,0.12728,0.12761,0
2022-05-16,HKD,CHF,0.12767,0.12820,0.12745,0.12760,1
2022-05-17,HKD,CHF,0.12759,0.12769,0.12634,0.12650,2
2022-05-18,HKD,CHF,0.12652,0.12719,0.12562,0.12581,3
2022-05-19,HKD,CHF,0.12584,0.12596,0.12356,0.12398,4


In [78]:
hkdcad = app.get_currency_exchange_daily('HKD', 'CAD')
hkdcad1 = hkdcad[0]
hkdcad1['to symbol'] = 'CAD'
hkdcad1['from symbol'] = 'HKD'

hkdcad1.rename(columns={'1. open': 'open'}, inplace=True)
hkdcad1.rename(columns={'2. high': 'high'}, inplace=True)
hkdcad1.rename(columns={'3. low': 'low'}, inplace=True)
hkdcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdcad1.index)
hkdcad1 = hkdcad1.sort_index()

hkdcad1['time'] = np.arange(len(hkdcad1.index))
hkdcad1 = hkdcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdcad1])

hkdcad1.to_pickle('hkdcad1.pkl')

hkdcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,CAD,0.16615,0.16616,0.16438,0.16441,0
2022-05-16,HKD,CAD,0.16446,0.16530,0.16348,0.16367,1
2022-05-17,HKD,CAD,0.16364,0.16371,0.16311,0.16313,2
2022-05-18,HKD,CAD,0.16314,0.16426,0.16298,0.16414,3
2022-05-19,HKD,CAD,0.16414,0.16414,0.16287,0.16336,4


In [79]:
hkdsek = app.get_currency_exchange_daily('HKD', 'SEK')
hkdsek1 = hkdsek[0]
hkdsek1['to symbol'] = 'SEK'
hkdsek1['from symbol'] = 'HKD'

hkdsek1.rename(columns={'1. open': 'open'}, inplace=True)
hkdsek1.rename(columns={'2. high': 'high'}, inplace=True)
hkdsek1.rename(columns={'3. low': 'low'}, inplace=True)
hkdsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdsek1.index)
hkdsek1 = hkdsek1.sort_index()

hkdsek1['time'] = np.arange(len(hkdsek1.index))
hkdsek1 = hkdsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdsek1])

hkdsek1.to_pickle('hkdsek1.pkl')

hkdsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,SEK,1.2859,1.2940,1.2789,1.2802,0
2022-05-16,HKD,SEK,1.2825,1.2859,1.2759,1.2793,1
2022-05-17,HKD,SEK,1.2794,1.2817,1.2572,1.2617,2
2022-05-18,HKD,SEK,1.2617,1.2812,1.2611,1.2787,3
2022-05-19,HKD,SEK,1.2785,1.2798,1.2592,1.2610,4


In [80]:
hkdnzd = app.get_currency_exchange_daily('HKD', 'NZD')
hkdnzd1 = hkdnzd[0]
hkdnzd1['to symbol'] = 'NZD'
hkdnzd1['from symbol'] = 'HKD'

hkdnzd1.rename(columns={'1. open': 'open'}, inplace=True)
hkdnzd1.rename(columns={'2. high': 'high'}, inplace=True)
hkdnzd1.rename(columns={'3. low': 'low'}, inplace=True)
hkdnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdnzd1.index)
hkdnzd1 = hkdnzd1.sort_index()

hkdnzd1['time'] = np.arange(len(hkdnzd1.index))
hkdnzd1 = hkdnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, hkdnzd1])

hkdnzd1.to_pickle('hkdnzd1.pkl')

hkdnzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,HKD,NZD,0.20413,0.20434,0.20229,0.20245,0
2022-05-16,HKD,NZD,0.20240,0.20419,0.20133,0.20163,1
2022-05-17,HKD,NZD,0.20160,0.20173,0.19965,0.19997,2
2022-05-18,HKD,NZD,0.20003,0.20224,0.19973,0.20216,3
2022-05-19,HKD,NZD,0.20206,0.20214,0.19839,0.19946,4


In [81]:
sekusd = app.get_currency_exchange_daily('SEK', 'USD')
sekusd1 = sekusd[0]
sekusd1['to symbol'] = 'USD'
sekusd1['from symbol'] = 'SEK'

sekusd1.rename(columns={'1. open': 'open'}, inplace=True)
sekusd1.rename(columns={'2. high': 'high'}, inplace=True)
sekusd1.rename(columns={'3. low': 'low'}, inplace=True)
sekusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekusd1.index)
sekusd1 = sekusd1.sort_index()

sekusd1['time'] = np.arange(len(sekusd1.index))
sekusd1 = sekusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekusd1])

sekusd1.to_pickle('sekusd1.pkl')

sekusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,USD,0.0980,0.0990,0.0979,0.0989,0
2022-05-16,SEK,USD,0.0988,0.0991,0.0985,0.0989,1
2022-05-17,SEK,USD,0.0988,0.1008,0.0988,0.1003,2
2022-05-18,SEK,USD,0.1003,0.1004,0.0989,0.0989,3
2022-05-19,SEK,USD,0.0990,0.1006,0.0989,0.1004,4


In [82]:
sekeur = app.get_currency_exchange_daily('SEK', 'EUR')
sekeur1 = sekeur[0]
sekeur1['to symbol'] = 'EUR'
sekeur1['from symbol'] = 'SEK'

sekeur1.rename(columns={'1. open': 'open'}, inplace=True)
sekeur1.rename(columns={'2. high': 'high'}, inplace=True)
sekeur1.rename(columns={'3. low': 'low'}, inplace=True)
sekeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekeur1.index)
sekeur1 = sekeur1.sort_index()

sekeur1['time'] = np.arange(len(sekeur1.index))
sekeur1 = sekeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekeur1])

sekeur1.to_pickle('sekeur1.pkl')

sekeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,EUR,0.09434,0.09506,0.09433,0.09495,0
2022-05-16,SEK,EUR,0.09491,0.09515,0.09462,0.09501,1
2022-05-17,SEK,EUR,0.09464,0.09548,0.09464,0.09502,2
2022-05-18,SEK,EUR,0.09505,0.09515,0.09443,0.09454,3
2022-05-19,SEK,EUR,0.09452,0.09490,0.09451,0.09469,4


In [83]:
sekjpy = app.get_currency_exchange_daily('SEK', 'JPY')
sekjpy1 = sekjpy[0]
sekjpy1['to symbol'] = 'JPY'
sekjpy1['from symbol'] = 'SEK'

sekjpy1.rename(columns={'1. open': 'open'}, inplace=True)
sekjpy1.rename(columns={'2. high': 'high'}, inplace=True)
sekjpy1.rename(columns={'3. low': 'low'}, inplace=True)
sekjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekjpy1.index)
sekjpy1 = sekjpy1.sort_index()

sekjpy1['time'] = np.arange(len(sekjpy1.index))
sekjpy1 = sekjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekjpy1])

sekjpy1.to_pickle('sekjpy1.pkl')

sekjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,JPY,12.6430,12.8560,12.6352,12.8380,0
2022-05-16,SEK,JPY,12.8005,12.8800,12.7291,12.8620,1
2022-05-17,SEK,JPY,12.7846,13.0829,12.7838,13.0139,2
2022-05-18,SEK,JPY,13.0228,13.0450,12.6969,12.7137,3
2022-05-19,SEK,JPY,12.7432,12.9030,12.7119,12.8396,4


In [84]:
sekgbp = app.get_currency_exchange_daily('SEK', 'GBP')
sekgbp1 = sekgbp[0]
sekgbp1['to symbol'] = 'GBP'
sekgbp1['from symbol'] = 'SEK'

sekgbp1.rename(columns={'1. open': 'open'}, inplace=True)
sekgbp1.rename(columns={'2. high': 'high'}, inplace=True)
sekgbp1.rename(columns={'3. low': 'low'}, inplace=True)
sekgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekgbp1.index)
sekgbp1 = sekgbp1.sort_index()

sekgbp1['time'] = np.arange(len(sekgbp1.index))
sekgbp1 = sekgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekgbp1])

sekgbp1.to_pickle('sekgbp1.pkl')

sekgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,GBP,0.08048,0.08108,0.08041,0.08080,0
2022-05-16,SEK,GBP,0.08074,0.08100,0.08040,0.08066,1
2022-05-17,SEK,GBP,0.08033,0.08081,0.08026,0.08043,2
2022-05-18,SEK,GBP,0.08044,0.08094,0.08023,0.08033,3
2022-05-19,SEK,GBP,0.08033,0.08063,0.08025,0.08051,4


In [85]:
sekaud = app.get_currency_exchange_daily('SEK', 'AUD')
sekaud1 = sekaud[0]
sekaud1['to symbol'] = 'AUD'
sekaud1['from symbol'] = 'SEK'

sekaud1.rename(columns={'1. open': 'open'}, inplace=True)
sekaud1.rename(columns={'2. high': 'high'}, inplace=True)
sekaud1.rename(columns={'3. low': 'low'}, inplace=True)
sekaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekaud1.index)
sekaud1 = sekaud1.sort_index()

sekaud1['time'] = np.arange(len(sekaud1.index))
sekaud1 = sekaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekaud1])

sekaud1.to_pickle('sekaud1.pkl')

sekaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,AUD,0.14352,0.14384,0.14309,0.14316,0
2022-05-16,SEK,AUD,0.14297,0.14418,0.14251,0.14294,1
2022-05-17,SEK,AUD,0.14234,0.14384,0.14199,0.14333,2
2022-05-18,SEK,AUD,0.14336,0.14354,0.14250,0.14289,3
2022-05-19,SEK,AUD,0.14282,0.14335,0.14254,0.14293,4


In [86]:
sekchf = app.get_currency_exchange_daily('SEK', 'CHF')
sekchf1 = sekchf[0]
sekchf1['to symbol'] = 'CHF'
sekchf1['from symbol'] = 'SEK'

sekchf1.rename(columns={'1. open': 'open'}, inplace=True)
sekchf1.rename(columns={'2. high': 'high'}, inplace=True)
sekchf1.rename(columns={'3. low': 'low'}, inplace=True)
sekchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekchf1.index)
sekchf1 = sekchf1.sort_index()

sekchf1['time'] = np.arange(len(sekchf1.index))
sekchf1 = sekchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekchf1])

sekchf1.to_pickle('sekchf1.pkl')

sekchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,CHF,0.09865,0.09961,0.0982,0.09924,0
2022-05-16,SEK,CHF,0.09890,0.10110,0.0985,0.09943,1
2022-05-17,SEK,CHF,0.09943,0.10046,0.0989,0.10003,2
2022-05-18,SEK,CHF,0.09960,0.10035,0.0976,0.09810,3
2022-05-19,SEK,CHF,0.09810,0.09873,0.0968,0.09760,4


In [87]:
sekcad = app.get_currency_exchange_daily('SEK', 'CAD')
sekcad1 = sekcad[0]
sekcad1['to symbol'] = 'CAD'
sekcad1['from symbol'] = 'SEK'

sekcad1.rename(columns={'1. open': 'open'}, inplace=True)
sekcad1.rename(columns={'2. high': 'high'}, inplace=True)
sekcad1.rename(columns={'3. low': 'low'}, inplace=True)
sekcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekcad1.index)
sekcad1 = sekcad1.sort_index()

sekcad1['time'] = np.arange(len(sekcad1.index))
sekcad1 = sekcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekcad1])

sekcad1.to_pickle('sekcad1.pkl')

sekcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,CAD,0.1275,0.1279,0.1271,0.1273,0
2022-05-16,SEK,CAD,0.1274,0.1278,0.1266,0.1271,1
2022-05-17,SEK,CAD,0.1268,0.1287,0.1266,0.1281,2
2022-05-18,SEK,CAD,0.1281,0.1282,0.1269,0.1271,3
2022-05-19,SEK,CAD,0.1272,0.1284,0.1270,0.1282,4


In [89]:
sekhkd = app.get_currency_exchange_daily('SEK', 'HKD')
sekhkd1 = sekhkd[0]
sekhkd1['to symbol'] = 'HKD'
sekhkd1['from symbol'] = 'SEK'

sekhkd1.rename(columns={'1. open': 'open'}, inplace=True)
sekhkd1.rename(columns={'2. high': 'high'}, inplace=True)
sekhkd1.rename(columns={'3. low': 'low'}, inplace=True)
sekhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekhkd1.index)
sekhkd1 = sekhkd1.sort_index()

sekhkd1['time'] = np.arange(len(sekhkd1.index))
sekhkd1 = sekhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, sekhkd1])

sekhkd1.to_pickle('sekhkd1.pkl')

sekhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,HKD,0.7704,0.7785,0.7701,0.7777,0
2022-05-16,SEK,HKD,0.7765,0.7800,0.7745,0.7800,1
2022-05-17,SEK,HKD,0.7769,0.7919,0.7769,0.7886,2
2022-05-18,SEK,HKD,0.7887,0.7895,0.7775,0.7780,3
2022-05-19,SEK,HKD,0.7780,0.7907,0.7780,0.7882,4


In [90]:
seknzd = app.get_currency_exchange_daily('SEK', 'NZD')
seknzd1 = seknzd[0]
seknzd1['to symbol'] = 'NZD'
seknzd1['from symbol'] = 'SEK'

seknzd1.rename(columns={'1. open': 'open'}, inplace=True)
seknzd1.rename(columns={'2. high': 'high'}, inplace=True)
seknzd1.rename(columns={'3. low': 'low'}, inplace=True)
seknzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(seknzd1.index)
seknzd1 = seknzd1.sort_index()

seknzd1['time'] = np.arange(len(seknzd1.index))
seknzd1 = seknzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, seknzd1])

seknzd1.to_pickle('seknzd1.pkl')

seknzd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,SEK,NZD,0.1530,0.1540,0.1526,0.1532,0
2022-05-16,SEK,NZD,0.1529,0.1541,0.1525,0.1530,1
2022-05-17,SEK,NZD,0.1523,0.1539,0.1522,0.1534,2
2022-05-18,SEK,NZD,0.1535,0.1535,0.1525,0.1530,3
2022-05-19,SEK,NZD,0.1529,0.1535,0.1521,0.1529,4


In [91]:
nzdusd = app.get_currency_exchange_daily('NZD', 'USD')
nzdusd1 = nzdusd[0]
nzdusd1['to symbol'] = 'USD'
nzdusd1['from symbol'] = 'NZD'

nzdusd1.rename(columns={'1. open': 'open'}, inplace=True)
nzdusd1.rename(columns={'2. high': 'high'}, inplace=True)
nzdusd1.rename(columns={'3. low': 'low'}, inplace=True)
nzdusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdusd1.index)
nzdusd1 = nzdusd1.sort_index()

nzdusd1['time'] = np.arange(len(nzdusd1.index))
nzdusd1 = nzdusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdusd1])

nzdusd1.to_pickle('nzdusd1.pkl')

nzdusd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,USD,0.62282,0.62897,0.6221,0.62820,0
2022-05-16,NZD,USD,0.62756,0.63204,0.6226,0.63090,1
2022-05-17,NZD,USD,0.63095,0.63757,0.6302,0.63598,2
2022-05-18,NZD,USD,0.63615,0.63701,0.6288,0.62944,3
2022-05-19,NZD,USD,0.62950,0.64172,0.6289,0.63810,4


In [92]:
nzdeur = app.get_currency_exchange_daily('NZD', 'EUR')
nzdeur1 = nzdeur[0]
nzdeur1['to symbol'] = 'EUR'
nzdeur1['from symbol'] = 'NZD'

nzdeur1.rename(columns={'1. open': 'open'}, inplace=True)
nzdeur1.rename(columns={'2. high': 'high'}, inplace=True)
nzdeur1.rename(columns={'3. low': 'low'}, inplace=True)
nzdeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdeur1.index)
nzdeur1 = nzdeur1.sort_index()

nzdeur1['time'] = np.arange(len(nzdeur1.index))
nzdeur1 = nzdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdeur1])

nzdeur1.to_pickle('nzdeur1.pkl')

nzdeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,EUR,0.59980,0.60406,0.59951,0.60300,0
2022-05-16,NZD,EUR,0.60364,0.60527,0.59910,0.60424,1
2022-05-17,NZD,EUR,0.60425,0.60743,0.60001,0.60248,2
2022-05-18,NZD,EUR,0.60250,0.60498,0.60070,0.60140,3
2022-05-19,NZD,EUR,0.60138,0.60693,0.59960,0.60270,4


In [93]:
nzdjpy = app.get_currency_exchange_daily('NZD', 'JPY')
nzdjpy1 = nzdjpy[0]
nzdjpy1['to symbol'] = 'JPY'
nzdjpy1['from symbol'] = 'NZD'

nzdjpy1.rename(columns={'1. open': 'open'}, inplace=True)
nzdjpy1.rename(columns={'2. high': 'high'}, inplace=True)
nzdjpy1.rename(columns={'3. low': 'low'}, inplace=True)
nzdjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdjpy1.index)
nzdjpy1 = nzdjpy1.sort_index()

nzdjpy1['time'] = np.arange(len(nzdjpy1.index))
nzdjpy1 = nzdjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdjpy1])

nzdjpy1.to_pickle('nzdjpy1.pkl')

nzdjpy1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,JPY,79.975,81.300,79.966,81.199,0
2022-05-16,NZD,JPY,81.201,81.598,80.164,81.444,1
2022-05-17,NZD,JPY,81.435,82.485,81.280,82.268,2
2022-05-18,NZD,JPY,82.290,82.434,80.628,80.759,3
2022-05-19,NZD,JPY,80.775,81.886,80.478,81.517,4


In [94]:
nzdgbp = app.get_currency_exchange_daily('NZD', 'GBP')
nzdgbp1 = nzdgbp[0]
nzdgbp1['to symbol'] = 'GBP'
nzdgbp1['from symbol'] = 'NZD'

nzdgbp1.rename(columns={'1. open': 'open'}, inplace=True)
nzdgbp1.rename(columns={'2. high': 'high'}, inplace=True)
nzdgbp1.rename(columns={'3. low': 'low'}, inplace=True)
nzdgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdgbp1.index)
nzdgbp1 = nzdgbp1.sort_index()

nzdgbp1['time'] = np.arange(len(nzdgbp1.index))
nzdgbp1 = nzdgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdgbp1])

nzdgbp1.to_pickle('nzdgbp1.pkl')

nzdgbp1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,GBP,0.51066,0.51311,0.51029,0.51200,0
2022-05-16,NZD,GBP,0.51254,0.51345,0.50880,0.51161,1
2022-05-17,NZD,GBP,0.51187,0.51335,0.50790,0.50887,2
2022-05-18,NZD,GBP,0.50880,0.51296,0.50820,0.50994,3
2022-05-19,NZD,GBP,0.50999,0.51325,0.50910,0.51132,4


In [96]:
nzdaud = app.get_currency_exchange_daily('NZD', 'AUD')
nzdaud1 = nzdaud[0]
nzdaud1['to symbol'] = 'AUD'
nzdaud1['from symbol'] = 'NZD'

nzdaud1.rename(columns={'1. open': 'open'}, inplace=True)
nzdaud1.rename(columns={'2. high': 'high'}, inplace=True)
nzdaud1.rename(columns={'3. low': 'low'}, inplace=True)
nzdaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdaud1.index)
nzdaud1 = nzdaud1.sort_index()

nzdaud1['time'] = np.arange(len(nzdaud1.index))
nzdaud1 = nzdaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdaud1])

nzdaud1.to_pickle('nzdaud1.pkl')

nzdaud1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,AUD,0.90824,0.90908,0.90370,0.90500,0
2022-05-16,NZD,AUD,0.90492,0.90764,0.90339,0.90441,1
2022-05-17,NZD,AUD,0.90450,0.90610,0.90210,0.90428,2
2022-05-18,NZD,AUD,0.90414,0.90699,0.90330,0.90453,3
2022-05-19,NZD,AUD,0.90447,0.90969,0.90290,0.90501,4


In [98]:
nzdchf = app.get_currency_exchange_daily('NZD', 'CHF')
nzdchf1 = nzdchf[0]
nzdchf1['to symbol'] = 'CHF'
nzdchf1['from symbol'] = 'NZD'

nzdchf1.rename(columns={'1. open': 'open'}, inplace=True)
nzdchf1.rename(columns={'2. high': 'high'}, inplace=True)
nzdchf1.rename(columns={'3. low': 'low'}, inplace=True)
nzdchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdchf1.index)
nzdchf1 = nzdchf1.sort_index()

nzdchf1['time'] = np.arange(len(nzdchf1.index))
nzdchf1 = nzdchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdchf1])

nzdchf1.to_pickle('nzdchf1.pkl')

nzdchf1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,CHF,0.62440,0.63025,0.62325,0.62936,0
2022-05-16,NZD,CHF,0.62824,0.63309,0.62465,0.63213,1
2022-05-17,NZD,CHF,0.63191,0.63482,0.62639,0.63185,2
2022-05-18,NZD,CHF,0.63184,0.63452,0.62022,0.62110,3
2022-05-19,NZD,CHF,0.62146,0.62553,0.61553,0.61936,4


In [99]:
nzdcad = app.get_currency_exchange_daily('NZD', 'CAD')
nzdcad1 = nzdcad[0]
nzdcad1['to symbol'] = 'CAD'
nzdcad1['from symbol'] = 'NZD'

nzdcad1.rename(columns={'1. open': 'open'}, inplace=True)
nzdcad1.rename(columns={'2. high': 'high'}, inplace=True)
nzdcad1.rename(columns={'3. low': 'low'}, inplace=True)
nzdcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdcad1.index)
nzdcad1 = nzdcad1.sort_index()

nzdcad1['time'] = np.arange(len(nzdcad1.index))
nzdcad1 = nzdcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdcad1])

nzdcad1.to_pickle('nzdcad1.pkl')

nzdcad1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,CAD,0.81250,0.81382,0.80849,0.81111,0
2022-05-16,NZD,CAD,0.81079,0.81348,0.80451,0.81073,1
2022-05-17,NZD,CAD,0.81076,0.81712,0.80964,0.81447,2
2022-05-18,NZD,CAD,0.81479,0.81671,0.81025,0.81093,3
2022-05-19,NZD,CAD,0.81099,0.82108,0.80956,0.81828,4


In [100]:
nzdhkd = app.get_currency_exchange_daily('NZD', 'HKD')
nzdhkd1 = nzdhkd[0]
nzdhkd1['to symbol'] = 'HKD'
nzdhkd1['from symbol'] = 'NZD'

nzdhkd1.rename(columns={'1. open': 'open'}, inplace=True)
nzdhkd1.rename(columns={'2. high': 'high'}, inplace=True)
nzdhkd1.rename(columns={'3. low': 'low'}, inplace=True)
nzdhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdhkd1.index)
nzdhkd1 = nzdhkd1.sort_index()

nzdhkd1['time'] = np.arange(len(nzdhkd1.index))
nzdhkd1 = nzdhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdhkd1])

nzdhkd1.to_pickle('nzdhkd1.pkl')

nzdhkd1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,HKD,4.8913,4.9357,4.8864,4.9318,0
2022-05-16,NZD,HKD,4.9331,4.9592,4.8898,4.9518,1
2022-05-17,NZD,HKD,4.9527,5.0009,4.9496,4.9930,2
2022-05-18,NZD,HKD,4.9914,4.9990,4.9370,4.9389,3
2022-05-19,NZD,HKD,4.9414,5.0328,4.9394,5.0057,4


In [101]:
nzdsek = app.get_currency_exchange_daily('NZD', 'SEK')
nzdsek1 = nzdsek[0]
nzdsek1['to symbol'] = 'SEK'
nzdsek1['from symbol'] = 'NZD'

nzdsek1.rename(columns={'1. open': 'open'}, inplace=True)
nzdsek1.rename(columns={'2. high': 'high'}, inplace=True)
nzdsek1.rename(columns={'3. low': 'low'}, inplace=True)
nzdsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdsek1.index)
nzdsek1 = nzdsek1.sort_index()

nzdsek1['time'] = np.arange(len(nzdsek1.index))
nzdsek1 = nzdsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

all_pairs = pd.concat([all_pairs, nzdsek1])

nzdsek1.to_pickle('nzdsek1.pkl')

nzdsek1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,NZD,SEK,6.2603,6.3367,6.2484,6.3244,0
2022-05-16,NZD,SEK,6.3121,6.3487,6.2786,6.3288,1
2022-05-17,NZD,SEK,6.3310,6.3572,6.2706,6.2917,2
2022-05-18,NZD,SEK,6.2914,6.3478,6.2914,6.3141,3
2022-05-19,NZD,SEK,6.3099,6.3686,6.3016,6.3124,4


View all_pairs dataframe:

In [102]:
all_pairs

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-05-13,USD,EUR,0.9629,0.9658,0.9596,0.9600,0
2022-05-16,USD,EUR,0.9612,0.9622,0.9572,0.9579,1
2022-05-17,USD,EUR,0.9579,0.9586,0.9471,0.9477,2
2022-05-18,USD,EUR,0.9476,0.9558,0.9464,0.9555,3
2022-05-19,USD,EUR,0.9552,0.9553,0.9426,0.9445,4
...,...,...,...,...,...,...,...
2022-09-23,NZD,SEK,6.4575,6.5108,6.4420,6.4690,95
2022-09-26,NZD,SEK,6.4208,6.5140,6.3617,6.3789,96
2022-09-27,NZD,SEK,6.3829,6.4421,6.3607,6.3928,97


Save to pickle:

In [103]:
all_pairs.to_pickle('all_pairs.pkl')

# Part Three
Obtain and clean sentiment data

Create empty dataframe to concatenate to

In [104]:
all_sentiment = pd.DataFrame()

* Request from API
* Get json

In [105]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:USD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

The data that was returned is very layered:
* Convert to list
* Pop unneeded items

In [106]:
listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

('relevance_score_definition',
 '0 < x <= 1, with a higher score indicating higher relevance.')

Flatten list twice (I found this worked the best):

In [107]:
flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

Explode 'ticker sentiment' column:

In [108]:
dftest = dftest.explode('ticker_sentiment')

* Delete unneeded columns
* Reset index

In [109]:
del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

Use a for loop to access dictionary inside 'ticker sentiment' column:
* Even though the API returns data for the specified ticker, it also returns duplicate rows of the same data for other tickers that the same news article also applies to
* I want to keep the data as organized as possible and do not want to deal with messy duplicates later after requesting from the API for other tickers
* The for loop checks if the ticker equals the specified currency - if not, the row is dropped

In [110]:
drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:USD':
                drop_values.append(k)

* Create new dataframe from list
* Sort values by overall sentiment score

In [111]:
usdforexsentiment = dftest.drop(index=drop_values)
usdforexsentiment = usdforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

* Add a column for symbol
* Set index to symbol

In [112]:
usdforexsentiment['symbol'] = 'USD'
usdforexsentiment.set_index('symbol', inplace=True)

* Concatenate to all_sentiment dataframe
* Check individual dataframe head to confirm everything worked

In [113]:
all_sentiment = pd.concat([all_sentiment, usdforexsentiment])

usdforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
USD,"Want to Save $50,000? Raise Your Credit Score",https://www.thestreet.com/personal-finance/wan...,20220929T132500,[Brian O'Connell],0.503978,Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Daily Turn or More,https://www.investorideas.com/news/2022/main/0...,20220929T143511,[],0.255236,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,VIA optronics AG ( VIAO ) Q2 2022 Earnings C...,https://www.fool.com/earnings/call-transcripts...,20220929T160032,[Motley Fool Transcribing],0.228677,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Lacroix : First half 2022 : Solid growth in re...,https://www.globenewswire.com/news-release/202...,20220929T160800,[Lacroix Group],0.227561,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Here's Why MLP ETFs Stood Out in Q3 Despite Oi...,https://www.zacks.com/stock/news/1986714/heres...,20220929T181700,[Zacks Investment Research],0.224804,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."


Save individual dataframe to pickle:

In [114]:
usdforexsentiment.to_pickle('usdforexsentiment.pkl')

Repeat for other currencies:

In [115]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:EUR&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:EUR':
                drop_values.append(k)

eurforexsentiment = dftest.drop(index=drop_values)
eurforexsentiment = eurforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

eurforexsentiment['symbol'] = 'EUR'
eurforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, eurforexsentiment])

eurforexsentiment.to_pickle('eurforexsentiment.pkl')

eurforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
EUR,Casino Group: Notes cancellation,https://www.globenewswire.com/news-release/202...,20220929T102900,[Groupe Casino],0.370555,Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,"Pound and euro need to fall further to reach ""...",https://www.reuters.com/markets/currencies/pou...,20220929T142100,[Reuters],0.335084,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,Interim Results,https://www.globenewswire.com/news-release/202...,20220929T081900,[Bango PLC],0.275904,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,Spain hopes to gain 3.14 billion euros in reve...,https://www.reuters.com/markets/europe/spain-h...,20220929T130500,[Reuters],0.245769,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,"The Zacks Analyst Blog Highlights Accenture, C...",https://www.zacks.com/stock/news/1986426/the-z...,20220929T120000,[Zacks Investment Research],0.245416,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."


In [116]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:JPY&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:JPY':
                drop_values.append(k)

jpyforexsentiment = dftest.drop(index=drop_values)
jpyforexsentiment = jpyforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

jpyforexsentiment['symbol'] = 'JPY'
jpyforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, jpyforexsentiment])

jpyforexsentiment.to_pickle('jpyforexsentiment.pkl')

jpyforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
JPY,World's biggest air-conditioner maker Daikin d...,https://www.business-standard.com/article/comp...,20220928T072100,[Reed Stevenson and Yuki Furukawa | Bloomberg],0.289199,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,Interim Results,https://www.globenewswire.com/news-release/202...,20220929T081900,[Bango PLC],0.275904,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,Warriors-Wizards games in Japan to woo NBA fan...,https://apnews.com/article/washington-wizards-...,20220929T063137,[],0.268941,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,Uno Minda forms JV with Japan's TACHI-S to exp...,https://www.financialexpress.com/express-mobil...,20220929T092711,[],0.264843,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,Tokyo Stocks Close Higher On US Gains,https://www.barrons.com/news/tokyo-stocks-clos...,20220929T061152,[AFP - Agence France Presse],0.262597,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."


In [118]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:GBP&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:GBP':
                drop_values.append(k)

gbpforexsentiment = dftest.drop(index=drop_values)
gbpforexsentiment = gbpforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

gbpforexsentiment['symbol'] = 'GBP'
gbpforexsentiment.set_index('symbol', inplace=True)
                            
all_sentiment = pd.concat([all_sentiment, gbpforexsentiment])
                            
gbpforexsentiment.to_pickle('gbpforexsentiment.pkl')

gbpforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
GBP,"Pound and euro need to fall further to reach ""...",https://www.reuters.com/markets/currencies/pou...,20220929T142100,[Reuters],0.335084,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,Is Amazon Stock a Buy After Its Massive New Pr...,https://www.fool.com/investing/2022/09/29/is-a...,20220929T123500,[Jeremy Bowman],0.287528,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,"The Zacks Analyst Blog Highlights Accenture, C...",https://www.zacks.com/stock/news/1986426/the-z...,20220929T120000,[Zacks Investment Research],0.245416,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,DT Midstream Increases Ownership in Millennium...,https://www.benzinga.com/pressreleases/22/09/g...,20220929T113000,[Globe Newswire],0.188909,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,DT Midstream Increases Ownership in Millennium...,https://www.globenewswire.com/news-release/202...,20220929T113000,"[DT Midstream, Inc.]",0.177437,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."


In [119]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:AUD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:AUD':
                drop_values.append(k)

audforexsentiment = dftest.drop(index=drop_values)
audforexsentiment = audforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

audforexsentiment['symbol'] = 'AUD'
audforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, audforexsentiment])

audforexsentiment.to_pickle('audforexsentiment.pkl')

audforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
AUD,Gold Fields ( GFI ) Reports Higher Revenues ...,https://www.zacks.com/stock/news/1973091/gold-...,20220826T200700,[Zacks Investment Research],0.347096,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,"Link Reit eyes more property in Australia, ent...",https://www.scmp.com/business/article/3193924/...,20220927T230008,[Cheryl Arcibal],0.320277,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Vail Resorts ( MTN ) Q4 2022 Earnings Call T...,https://www.fool.com/earnings/call-transcripts...,20220929T000032,[Motley Fool Transcribing],0.286237,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Private equity giant Thoma Bravo acquires Aust...,https://www.fnlondon.com/articles/private-equi...,20220823T170919,[Stuart Condie],0.268585,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Verint Systems ( VRNT ) Q2 2023 Earnings Cal...,https://www.fool.com/earnings/call-transcripts...,20220908T033023,[Motley Fool Transcribing],0.262093,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."


In [120]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:CHF&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:CHF':
                drop_values.append(k)

chfforexsentiment = dftest.drop(index=drop_values)
chfforexsentiment = chfforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

chfforexsentiment['symbol'] = 'CHF'
chfforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, chfforexsentiment])

chfforexsentiment.to_pickle('chfforexsentiment.pkl')

chfforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
CHF,"Holcim closes Ambuja Cement sale to Adani, get...",https://www.business-standard.com/article/comp...,20220916T071100,[Dev Chatterjee],0.421127,Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Credit Suisse ( CS ) to Own 100% of China Se...,https://www.zacks.com/stock/news/1979322/credi...,20220912T125900,[Zacks Investment Research],0.291632,Somewhat-Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Zaya Younan Buys His Third Vineyard in Saint-É...,https://www.benzinga.com/pressreleases/22/09/g...,20220906T140000,[Globe Newswire],0.290667,Somewhat-Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Vail Resorts ( MTN ) Q4 2022 Earnings Call T...,https://www.fool.com/earnings/call-transcripts...,20220929T000032,[Motley Fool Transcribing],0.286237,Somewhat-Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Destination Palisade: Don't miss Colorado harv...,https://www.axios.com/local/denver/2022/09/03/...,20220903T120058,[https://www.axios.com/local/denver/authors/jf...,0.282596,Somewhat-Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."


In [121]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:CAD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:CAD':
                drop_values.append(k)

cadforexsentiment = dftest.drop(index=drop_values)
cadforexsentiment = cadforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

cadforexsentiment['symbol'] = 'CAD'
cadforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, cadforexsentiment])

cadforexsentiment.to_pickle('cadforexsentiment.pkl')

cadforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
CAD,Fidelity Investments Canada Launches New Fidel...,https://www.newswire.ca/news-releases/fidelity...,20220928T113000,[Fidelity Investments Canada ULC],0.506182,Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,Fidelity Investments Canada Launches New Fidel...,https://www.benzinga.com/pressreleases/22/09/n...,20220928T113000,[PRNewswire],0.489520,Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,LSEG Integrates CanDeal Pricing into Its Canad...,https://www.newswire.ca/news-releases/lseg-int...,20220921T130000,[CanDeal],0.300293,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,Vail Resorts ( MTN ) Q4 2022 Earnings Call T...,https://www.fool.com/earnings/call-transcripts...,20220929T000032,[Motley Fool Transcribing],0.286237,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,"Bitcoin Well Announces Customer Milestone, New...",https://www.globenewswire.com/news-release/202...,20220920T120000,[Bitcoin Well],0.285458,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."


In [122]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:HKD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:HKD':
                drop_values.append(k)

hkdforexsentiment = dftest.drop(index=drop_values)
hkdforexsentiment = hkdforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

hkdforexsentiment['symbol'] = 'HKD'
hkdforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, hkdforexsentiment])

hkdforexsentiment.to_pickle('hkdforexsentiment.pkl')

hkdforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
HKD,Futu Holdings Ltd ( FUTU ) Q2 2022 Earnings ...,https://www.fool.com/earnings/call-transcripts...,20220830T150025,[Motley Fool Transcribing],0.235365,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,US dollar strength lets Hong Kong investors go...,https://www.scmp.com/comment/opinion/article/3...,20220913T080020,[Neal Kimberley],0.232905,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,Exclusive: HKMA trials to find suitable use fo...,https://www.scmp.com/business/banking-finance/...,20220929T003022,"[Enoch Yiu, Peggy Sito]",0.216266,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,Hong Kong equities beat US stock picks in inve...,https://www.scmp.com/business/markets/article/...,20220908T082540,[Enoch Yiu],0.213054,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,"Hong Kong lawmakers, brokers support yuan-deno...",https://www.scmp.com/business/markets/article/...,20220908T093011,"[Georgina Lee, Enoch Yiu]",0.202372,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."


In [123]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:SEK&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:SEK':
                drop_values.append(k)

sekforexsentiment = dftest.drop(index=drop_values)
sekforexsentiment = sekforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

sekforexsentiment['symbol'] = 'SEK'
sekforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, sekforexsentiment])

sekforexsentiment.to_pickle('sekforexsentiment.pkl')

sekforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
SEK,Kvika banki hf.: Announcement regarding increa...,https://www.globenewswire.com/news-release/202...,20220922T162900,[Kvika banki hf.],0.378631,Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,Kvika banki hf.: Announcement regarding increa...,https://www.globenewswire.com/news-release/202...,20220818T173000,[Kvika banki hf.],0.374410,Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,Worldline - H1 2022 results,https://www.globenewswire.com/news-release/202...,20220727T050000,[WORLDLINE SA],0.345460,Somewhat-Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,Can Colgate's ( CL ) Growth Efforts Offset C...,https://www.zacks.com/stock/news/1950515/can-c...,20220711T135500,[Zacks Investment Research],0.340412,Somewhat-Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,Enento Group's Half Year Financial Report 1.1....,https://www.globenewswire.com/news-release/202...,20220721T080000,[Enento Group Oyj],0.303004,Somewhat-Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."


In [124]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:NZD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:NZD':
                drop_values.append(k)

nzdforexsentiment = dftest.drop(index=drop_values)
nzdforexsentiment = nzdforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

nzdforexsentiment['symbol'] = 'NZD'
nzdforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, nzdforexsentiment])

nzdforexsentiment.to_pickle('nzdforexsentiment.pkl')

nzdforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
NZD,EverCommerce Inc. ( EVCM ) Q2 2022 Earnings ...,https://www.fool.com/earnings/call-transcripts...,20220809T090025,[Motley Fool Transcribing],0.340220,Somewhat-Bullish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,New Zealand Central Bank Serves Up 4th Straigh...,https://www.benzinga.com/markets/asia/22/08/28...,20220817T112913,[Bhavik Nair],0.127383,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Rupee remains best performer against other glo...,https://www.financialexpress.com/market/rupee-...,20220911T035021,[],0.053979,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Speculators cut long U.S. dollar bets in lates...,https://www.reuters.com/markets/us/speculators...,20220812T193900,[Reuters],0.052987,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Major currencies hold steady ahead of Fed minu...,https://www.reuters.com/markets/us/major-curre...,20220817T015100,[Alun John],0.052572,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."


View all_sentiment dataframe:

In [125]:
all_sentiment

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
USD,"Want to Save $50,000? Raise Your Credit Score",https://www.thestreet.com/personal-finance/wan...,20220929T132500,[Brian O'Connell],0.503978,Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Daily Turn or More,https://www.investorideas.com/news/2022/main/0...,20220929T143511,[],0.255236,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,VIA optronics AG ( VIAO ) Q2 2022 Earnings C...,https://www.fool.com/earnings/call-transcripts...,20220929T160032,[Motley Fool Transcribing],0.228677,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Lacroix : First half 2022 : Solid growth in re...,https://www.globenewswire.com/news-release/202...,20220929T160800,[Lacroix Group],0.227561,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Here's Why MLP ETFs Stood Out in Q3 Despite Oi...,https://www.zacks.com/stock/news/1986714/heres...,20220929T181700,[Zacks Investment Research],0.224804,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
...,...,...,...,...,...,...,...
NZD,Euro on thin ice ahead of U.S. labour data,https://www.reuters.com/markets/us/euro-thin-i...,20220708T020400,[Tom Westbrook],-0.180598,Somewhat-Bearish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Japanese yen rises as investors seek havens af...,https://www.aljazeera.com/economy/2022/7/8/jap...,20220708T045733,[],-0.204529,Somewhat-Bearish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,"Dollar gains with yen, Aussie drops as investo...",https://www.reuters.com/markets/europe/dollar-...,20220701T053800,[Kevin Buckland],-0.215152,Somewhat-Bearish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."


Save to pickle:

In [126]:
all_sentiment.to_pickle('all_sentiment.pkl')